# Netherlands Neurogenetics Database
Author: Nienke Mekkes <br>
Date: 17-01-2024. <br>
Correspond: n.j.mekkes@umcg.nl <br>

## Script: Seurat dimensionality reduction
Objectives: Implementation of Seurat dimensionality reduction <br>

#### Minimal requirements

In [ ]:
# Enter commands in R (or R studio, if installed)
# install.packages('Seurat')

#####
library(Seurat)
library(dplyr)
library(patchwork)

library(ggplot2) 
library(stringr)
library(data.table)
library(readxl)
library(viridis)



In [ ]:
sessionInfo()

## Loading custom colour dictionaries
We will use specific colours for specific diagnoses/categories. All these colours are predefined in the colours_seurat.R script.

In [ ]:
source("colours_seurat.R")

Small example:

In [ ]:
custom_colors_origin

## Custom functions
Custom functions for:  
- Preprocessing seurat objects  
- Calculating p-values with chi-squared test. 
- Creating different kinds of plots
- Finding significant markers
- Performing PCA
- Performing coherent analysis 

These functions are encompassed in a single R script called functions_seurat.R

In [ ]:
source("functions_seurat.R")

#### load in sup3, which contains symptom domain ordering

In [ ]:
path_to_sup3 <- "/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/input_data/sup3.xlsx"
sup3 <- read_excel(path_to_sup3)
sup3 <- sup3 %>%
  mutate(ITname = str_replace_all(ITname, "_", "-"))

head(sup3)

#### load in PRS scores, can be added to analysis

In [ ]:
path_to_prs <- "/home/jupyter-n.mekkes@gmail.com-f6d87/genetics/analyse_PRS/PRS_combined_zscore.xlsx"
prs_scores <- read_excel(path_to_prs)
# prs_scores

#### load in rules of thumb predictions

In [ ]:
# df = read.table('/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_diagnosis/output/selected_diagnoses_july.csv', sep=',', header= TRUE)
df = read.table('/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_analysis/data/seurat_input.csv', sep=',', header= TRUE)
# # colnames(df)
non_attributes <- c('neuropathological_diagnosis','age_at_death','Year','sex','DonorID','Age','file_year','simplified_diagnosis')
attributes <- colnames(df)[!(colnames(df) %in% non_attributes)] ## the symptom names
metadata_columns <- c("DonorID","age_at_death", "sex","neuropathological_diagnosis",'simplified_diagnosis',"MS","AD","FTD","file_year"#,
                      # 'parsed_clinical_diagnosis', 'diagnosis_info', 'pred', 'pred_info', 'model_vs_clinic'
                     ) #diagnosis_info
selected_df <- df[, c("DonorID", "simplified_diagnosis")]
selected_df <- unique(selected_df)
diagnosis_counts <- table(selected_df$simplified_diagnosis)
print(diagnosis_counts)
# print(head(df))

head(df)
length(unique(df$DonorID))

table(df$Age)

In [ ]:
jig <- df[df$simplified_diagnosis == 'other_dem', ]
table(jig$neuropathological_diagnosis)

In [ ]:

# df <- df[df$Age != -9,]
# 
## group some diagnoses together
# df <- df %>%
#   mutate(neuropathological_diagnosis = ifelse(neuropathological_diagnosis == "PDD", "PD", neuropathological_diagnosis))
df2 <- df %>%
  mutate(simplified_diagnosis = ifelse(simplified_diagnosis == "PDD", "PD",
                                             ifelse(simplified_diagnosis %in% c("MDD", "BP", "SCZ","other_psych"), "PSYCH",
                                                    simplified_diagnosis)))

# df$years_before_death <- df$age_at_death - df$Age

# unique_values <- unique(df$simplified_diagnosis)
# print(unique_values)
# table1 <- c('CON', 'FTD','PD', 'AD', 'PSP', 'MS', 'VD' ,'ATAXIA','MSA', 'MND' ,'DLB','PSYCH')
# table1 <- c('CON', 'FTD','PD', 'AD', 'PSP', 'MS', 'VD' ,'ATAXIA','MSA', 'MND' ,'DLB',"MDD", "BP", "SCZ")
df2 <- merge(df2, prs_scores, by.x = "DonorID", by.y = "DonorID", all.x = TRUE)
df2$MS <- replace(df2$MS, is.na(df2$MS), 'exclude')
df2$AD <- replace(df2$AD, is.na(df2$AD), 'exclude')
df2$FTD <- replace(df2$FTD, is.na(df$FTD), 'exclude')
# df$neuropathological_diagnosis <- ifelse(df$neuropathological_diagnosis %in% table1, df$neuropathological_diagnosis, 'other')
selected_df <- df2[, c("DonorID", "simplified_diagnosis")]
selected_df <- unique(selected_df)
diagnosis_counts <- table(selected_df$simplified_diagnosis)
print(diagnosis_counts)

selected_df <- df2[, c("DonorID", "neuropathological_diagnosis")]
selected_df <- unique(selected_df)
diagnosis_counts <- table(selected_df$neuropathological_diagnosis)
print(diagnosis_counts)
new_column_names <- str_replace_all(attributes, "_", "-")

# Rename the columns in the data frame
df2 <- df2 %>%
  rename_with(~ new_column_names, all_of(attributes))


# table(df$Age)
############### remove later #############
# df <- df %>%
#   filter(neuropathological_diagnosis != "other")

# df$file_year <- gsub("NBB (\\d{4})-\\d{3}", "\\1", df$DonorID)
# df$file_year <- as.numeric(df$file_year)
# df <- subset(df, file_year >= 1997)
dim(df)
dim(df2)
head(df)
head(df2)

#### Data preprocessing: metadata

In [ ]:
########## METADATA ##################
meta_data <- df2[, metadata_columns]

age_bins <- c("0-20", "20-40", "40-60", "60-80", "80-100", "100+")
clin_grud_overview <- read_excel('/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_analysis/data/grud_clin_subset_overview_both.xlsx')
clin_grud_overview <- read_excel('/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_analysis/data/grud_clin_subset_overview_newboth.xlsx')
gen_info <- read_excel('/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/input_data/General_information_11-08-2023.xlsx')

# clin_grud_overview$FTD_misdiagnosis <- ifelse(clin_grud_overview$neuropathological_diagnosis == "FTD" & clin_grud_overview$diagnosis_info == 'non-coherent', 1, 0)
print(dim(clin_grud_overview))
# diagnosis_counts <- table(clin_grud_overview$FTD_misdiagnosis)
# print(diagnosis_counts)
print(dim(meta_data))
meta_data <- merge(meta_data, clin_grud_overview[, c('DonorID', 'parsed_clinical_diagnosis', 'clin_coherence')],
                   by='DonorID', all.x=TRUE)
meta_data <- merge(meta_data, gen_info[, c('DonorID', 'APOE status')],
                   by='DonorID', all.x=TRUE)
columns_to_replace <- c('parsed_clinical_diagnosis','clin_coherence')
# for (col in columns_to_replace) {
#   meta_data[[col]] <- ifelse(meta_data$DonorID %in% clin_grud_overview$DonorID, meta_data[[col]], "unknown/coherent")
# }
# order_levels <- c("coherent", "ambiguous", "non-coherent", "unknown")
# meta_data$diagnosis_info <- factor(meta_data$diagnosis_info, levels = order_levels)
# meta_data$clin_coherence <- ifelse(is.na(meta_data$clin_coherence), "unknown", df$clin_coherence)
meta_data$clin_coherence <- ifelse(is.na(meta_data$clin_coherence), "unknown/coherent", meta_data$clin_coherence)

order_levels <- c("unknown/coherent","non_coherent")
meta_data$clin_coherence <- factor(meta_data$clin_coherence, levels = order_levels)
# Replace the values in the diagnosis_info column with the desired order


# ## set up some options to assess the ellips
# meta_data <- meta_data %>%
#   mutate(model_and_clinic1 =
#            case_when(
#              diagnosis_info == "ambiguous" & pred_info == "non-coherent" ~ "non-coherent",
#              diagnosis_info == "non-coherent" & pred_info == "non-coherent" ~ "non-coherent",
#              diagnosis_info == "unknown" | pred_info == "unknown" ~ "unknown",
#              TRUE ~ "coherent"
#            ))
# meta_data <- meta_data %>%
#   mutate(model_and_clinic2 =
#            case_when(
#              # (diagnosis_info == "ambiguous" & pred_info == "non-coherent") |
#              (diagnosis_info == "non-coherent" & pred_info == "non-coherent") ~ "non-coherent",
#              diagnosis_info == "unknown" | pred_info == "unknown" ~ "unknown",
#              TRUE ~ "coherent"
#            ))
# meta_data <- meta_data %>%
#   mutate(model_and_clinic3 =
#            case_when(
#              diagnosis_info == "non-coherent" & pred_info == "ambiguous" ~ "non-coherent",
#              diagnosis_info == "non-coherent" & pred_info == "non-coherent" ~ "non-coherent",
#              diagnosis_info == "unknown" | pred_info == "unknown" ~ "unknown",
#              TRUE ~ "coherent"
#            ))
# meta_data <- meta_data %>%
#   mutate(model_and_clinic4 =
#            case_when(
#              diagnosis_info == "ambiguous" & pred_info == "non-coherent" ~ "non-coherent",
#              diagnosis_info == "ambiguous" & pred_info == "ambiguous" ~ "non-coherent",
#              diagnosis_info == "non-coherent" & pred_info == "ambiguous" ~ "non-coherent",  
#              diagnosis_info == "non-coherent" & pred_info == "non-coherent" ~ "non-coherent",
#              diagnosis_info == "unknown" | pred_info == "unknown" ~ "unknown",
#              TRUE ~ "coherent"
#            ))
# meta_data <- meta_data %>%
#   mutate(clin1 =
#            case_when(
#              diagnosis_info == "non-coherent" ~ "non-coherent",
#              diagnosis_info == "unknown" ~ "unknown",
#              TRUE ~ "coherent"
#            ))
# meta_data <- meta_data %>%
#   mutate(clin2 =
#            case_when(
#              diagnosis_info == "coherent" ~ "coherent",
#              diagnosis_info == "unknown" ~ "unknown",
#              TRUE ~ "non-coherent"
#            ))
meta_data <- meta_data %>% distinct()
tail(meta_data,5)
print(dim(meta_data))
# print(table(meta_data[, c("DonorID", "model_and_clinic1")]$model_and_clinic1))
# print(table(meta_data[, c("DonorID", "model_and_clinic2")]$model_and_clinic2))
# print(table(meta_data[, c("DonorID", "model_and_clinic3")]$model_and_clinic3))
# print(table(meta_data[, c("DonorID", "model_and_clinic4")]$model_and_clinic4))
# print(table(meta_data[, c("DonorID", "clin1")]$clin1))
# print(table(meta_data[, c("DonorID", "clin2")]$clin2))
# print(table(meta_data[, c("DonorID", "diagnosis_info")]$diagnosis_info))
# subset(meta_data, model_and_clinic1 == "other")
# print(table(subset(meta_data, model_and_clinic == "other")[, c("DonorID", "diagnosis_info")]$diagnosis_info))
# # Replace numeric age_bin values with corresponding age bin labels
# meta_data$model_and_clinic1 <- factor(meta_data$model_and_clinic1, levels = order_levels)
# meta_data$model_and_clinic2 <- factor(meta_data$model_and_clinic2, levels = order_levels)
# meta_data$model_and_clinic3 <- factor(meta_data$model_and_clinic3, levels = order_levels)
# meta_data$model_and_clinic4 <- factor(meta_data$model_and_clinic4, levels = order_levels)
# meta_data$clin1 <- factor(meta_data$clin1, levels = order_levels)
# meta_data$clin2 <- factor(meta_data$clin2, levels = order_levels)

meta_data$age_bin <- cut(meta_data$age_at_death, breaks = seq(0, 120, by = 20),
                         labels = age_bins, include.lowest = TRUE, right = FALSE)
# meta_data$AD_misdiagnosis <- ifelse(meta_data$simplified_diagnosis == "AD" & meta_data$diagnosis_info == 'non-coherent', 1, 0)
# meta_data$FTD_misdiagnosis <- ifelse(meta_data$simplified_diagnosis == "FTD" & meta_data$diagnosis_info == 'non-coherent', 1, 0)
# meta_data$PD_misdiagnosis <- ifelse(meta_data$simplified_diagnosis == "PD" & meta_data$diagnosis_info == 'non-coherent', 1, 0)
# meta_data$VD_misdiagnosis <- ifelse(meta_data$simplified_diagnosis == "VD" & meta_data$diagnosis_info == 'non-coherent', 1, 0)
# meta_data$ATAXIA_misdiagnosis <- ifelse(meta_data$simplified_diagnosis == "ATAXIA" & meta_data$diagnosis_info =='non-coherent', 1, 0)
# meta_data$DLB_misdiagnosis <- ifelse(meta_data$simplified_diagnosis == "DLB" & meta_data$diagnosis_info == 'non-coherent', 1, 0)
# meta_data$MND_misdiagnosis <- ifelse(meta_data$simplified_diagnosis == "MND" & meta_data$diagnosis_info == 'non-coherent', 1, 0)
# meta_data$PSP_misdiagnosis <- ifelse(meta_data$simplified_diagnosis == "PSP" & meta_data$diagnosis_info == 'non-coherent', 1, 0)
# meta_data$MS_misdiagnosis <- ifelse(meta_data$simplified_diagnosis == "MS" & meta_data$diagnosis_info == 'non-coherent', 1, 0)
# meta_data$MSA_misdiagnosis <- ifelse(meta_data$simplified_diagnosis == "MSA" & meta_data$diagnosis_info == 'non-coherent', 1, 0)

meta_data <- unique(meta_data)
# mania_metadata <- meta_data
row.names(meta_data) <- meta_data$DonorID
meta_data$DonorID <- NULL
table(meta_data$clin_coherence)

In [ ]:
head(meta_data)

#### Data preprocessing: flattened observations

In [ ]:
# flattened_data <- df %>%
#   select(-symptoms, -groupings, -Domains)

flattened_data <- df2 %>%
  group_by(across(all_of(metadata_columns))) %>%
  summarise(across(everything(), sum))

flattened_data <- as.data.frame(flattened_data)
rownames(flattened_data) <- flattened_data$DonorID
flattened_data <- flattened_data %>%
  select(all_of(c("DonorID", new_column_names)))


tail(flattened_data)
dim(flattened_data)

#### Data preprocessing: binned observations

In [ ]:
print(dim(df2))
df2 <- df2[df2$Age >= 0, ]
print(dim(df2))
######## BINS of AGE ###############
custom_start_age <- 15
custom_end_age <- 105
bin_width <- 30
overlap <- 5
max_bins <- max(df2$Age)
num_bins <- floor((max_bins - bin_width) / overlap) + 1
num_bins <- floor((custom_end_age - custom_start_age - bin_width) / overlap) + 1

reshaped_data <- data.frame(DonorID = df2$DonorID, Age = df2$Age)

print(num_bins)
# for (symptom in attributes) {
#   for (i in 1:num_bins) {
#       start_age <- (i-1)*overlap
#       end_age <- start_age + bin_width
#       col_name <- paste0(symptom, ".", start_age, ".", end_age)
#       reshaped_data[col_name] <- ifelse(df$Age >= start_age & df$Age < end_age & df[[symptom]] == 1, 1, 0)
#   }
# }

for (symptom in new_column_names) {
  for (i in 1:num_bins) {
      start_age <- custom_start_age + (i - 1) * overlap
      end_age <- start_age + bin_width
      col_name <- paste0(symptom, ".", start_age, ".", end_age)
      reshaped_data[col_name] <- ifelse(df2$Age >= start_age & df2$Age < end_age & df2[[symptom]] == 1, 1, 0)
  }
}

grouped_data <- reshaped_data %>%
  group_by(DonorID) %>%
  summarise(across(where(is.numeric), sum))

grouped_data <- grouped_data[, -which(names(grouped_data) == "Age")]
age_reshaped <- as.data.frame(grouped_data)
row.names(age_reshaped) <- age_reshaped$DonorID
print(dim(age_reshaped))
# head(age_reshaped,10)
############################################


####### TERUG VANAF DOOD ########
# reshaped_data_death <- data.frame(DonorID = df$DonorID, Age = df$Age)
# bin_width <- 30
# overlap <- 15
# max_years_before_death <- max(df$years_before_death)
# num_bins <- floor((max_years_before_death - bin_width) / overlap) + 1
# for (symptom in attributes) {
#   for (i in 1:num_bins) {
#     bin_start <- (i - 1) * overlap
#     bin_end <- bin_start + bin_width
#     col_name <- paste0(symptom, ".", bin_start, ".", bin_end,'BD')
#     reshaped_data_death[col_name] <- ifelse(df$years_before_death >= bin_start & df$years_before_death < bin_end & df[[symptom]] == 1, 1, 0)
#   }
# }
# grouped_data_death <- reshaped_data_death %>%
#   group_by(DonorID) %>%
#   summarise(across(where(is.numeric), sum)) 
# reshaped_data_death <- as.data.frame(grouped_data_death)
# row.names(reshaped_data_death) <- reshaped_data_death$DonorID
############################################

########### combineer age met death #####################
# merged_data <- merge(age_reshaped, reshaped_data_death, by = "DonorID", all = TRUE)
# rownames(merged_data) <- merged_data$DonorID ##set rownames



In [ ]:
# # head(age_reshaped)
# # head(mania_metadata)
# sum(age_reshaped$Mania.15.45)
# mania_inspect <- age_reshaped %>%
#   left_join(mania_metadata %>% 
#               select(DonorID, neuropathological_diagnosis), 
#             by = "DonorID")

# # head
# result <- mania_inspect %>%
#   group_by(neuropathological_diagnosis) %>%
#   summarise(total_Mania = sum(Mania.15.45))

# # Print the result
# result

In [ ]:
########## combineer age met flattened #####################
merged_data <- merge(age_reshaped, flattened_data, by = "DonorID", all = TRUE)
rownames(merged_data) <- merged_data$DonorID ##set rownames

########### or not #####################
# merged_data <- age_reshaped 
# merged_data <- flattened_data 

merged_data$DonorID <- NULL
merged_data$Age <-NULL
head(merged_data,10)

flat_features <- colnames(flattened_data)
flat_features <- flat_features[flat_features != "DonorID"]
# quant_features <- as.character(quant_features)
head(flat_features)

temp_features <- colnames(age_reshaped)
temp_features <- temp_features[temp_features != "DonorID"]
# quant_features <- as.character(quant_features)
head(temp_features)

In [ ]:
flattened_data$DonorID <- NULL
flattened_data$Age <-NULL

age_reshaped$DonorID <- NULL
age_reshaped$Age <-NULL

In [ ]:
dim(flattened_data)
dim(age_reshaped)

In [ ]:
# hist(merged_data$`Dementia.60.90`)
# most_observations <- names(colSums(merged_data))[which.max(colSums(merged_data))]
# print(most_observations)
# hist(merged_data[[most_observations]],main=most_observations)

most_observations_flattened <- names(colSums(flattened_data))[which.max(colSums(flattened_data))]
print(most_observations_flattened)
hist(flattened_data[[most_observations_flattened]],main=most_observations_flattened)

most_observations_temporal <- names(colSums(age_reshaped))[which.max(colSums(age_reshaped))]
print(most_observations_temporal)
hist(age_reshaped[[most_observations_temporal]],main=most_observations_temporal)

#### correlation analysis

In [ ]:
# colnames(flattened_data)

In [ ]:
# foo <- c('Mobility-problems','Rigidity','Vertigo','Dementia','Cognitive-decline','Bradyphrenia','Admission-to-nursing-home')

# for (symptom in foo) {
#   # Get the columns corresponding to the current symptom
#   columns <- age_reshaped[grep(paste0("^", symptom), names(age_reshaped))]
  
#   # Calculate correlations and p-values
#   cor_p_values <- sapply(columns, function(column) {
#     cor_test_result <- cor.test(flattened_data$Dementia, column)
#     return(list(correlation = cor_test_result$estimate, p_value = cor_test_result$p.value))
#   })
  
#   correlations <- cor_p_values[1, ]
#   p_values <- cor_p_values[2, ]
  
#   # Take the log of p_values
#   log_p_values <- log10(as.numeric(p_values))
  
#   # Create a data frame for the current symptom
#   correlation_df <- data.frame(cor = unlist(correlations), p = unlist(log_p_values))
#   rownames(correlation_df) <- gsub("cor$", "", rownames(correlation_df))
#   # print(correlation_df)
#   # Plot for the current symptom
#   library(ggplot2)
#   plot <- ggplot(correlation_df, aes(x = cor, y = p)) +
#     geom_point() +
#     geom_errorbar(aes(ymin = p, ymax = p)) +
#     geom_text(aes(label = rownames(correlation_df)), vjust = -0.5, hjust = 0.5) +  # Add symptom names as labels
#     xlab("Correlation") +
#     ylab("log10 p-value") +
#     theme_minimal() +
#     ggtitle(paste("Correlation vs. log10 p-value for", symptom))
  
#   # Print the plot
#   print(plot)
# }

#### symptom (gene) metadata

In [ ]:
merged_data = t(merged_data)
flattened_data = t(flattened_data)
age_reshaped = t(age_reshaped)
# symptom_metadata <- data.frame(gene_short_name = rownames(merged_data))
# rownames(symptom_metadata) <- rownames(merged_data)
# head(symptom_metadata)

In [ ]:
dim(flattened_data)
dim(age_reshaped)

#### seurat

- Data structure creation: <br>
  It creates a new Seurat object and initializes its data structure to 
  store the symptombin observation data and donor metadata

- Count matrix assignment: <br>
  It assigns the symptombin observation counts to the Seurat object.<br>

- Quality control (QC): <br>
  It performs basic QC checks on the input data, such as filtering out donors with low quality or low symptombin observation counts. <br>
  The min.cells and min.features arguments you provide specify the minimum number of donors and symptombins required for a donor to be included in the analysis.

- Normalization: <br>
  It applies normalization methods to the count matrix to account for differences in sequencing depth and other technical factors. <br>
  The specific normalization method used depends on the default settings or any additional arguments you provide. we will do this in a later step to keep track

- Feature selection: <br>
  It may perform feature selection or gene filtering to focus on the most informative genes for downstream analysis.<br>
  This step helps reduce noise and dimensionality in the dataset.

- Metadata assignment:<br>
  It sets up the initial metadata slots in the Seurat object, including information about the project, cell attributes, and other relevant annotations. <br>
  You can subsequently add or modify metadata using the provided accessors and assignment operators.


In [ ]:
# ## min.cells: 
# ## Include features detected in at least this many cells.
# ## e.g. includes symptombins that are detected in at least 5 donors (if a symptombin is even rarer, it might not be relevant)

# ## min.features: 
# ## Include cells where at least this many features are detected.
# ## e.g. includes donors where at least 1 features are detected (control donors might have zero features,
# ## but if a donor has zero features, we dont know where to cluster them!)
# flattened_seurat <- CreateSeuratObject(counts = flattened_data, project = "NND", min.cells = 5, min.features = 1)
# flattened_seurat <- preprocess_seurat_object(flattened_seurat,meta_data)
# ## flattened has a donor that temporal doenst have, we remove this one
# toRemove <- c('NBB 1998-116')
# flattened_seurat <- flattened_seurat[,!colnames(flattened_seurat) %in% toRemove]
# # flattened_seurat
# flattened_seurat

In [ ]:
# temporal_seurat <- CreateSeuratObject(counts = age_reshaped, project = "NND", min.cells = 5, min.features = 1)
# temporal_seurat <- preprocess_seurat_object(temporal_seurat,meta_data)
# temporal_seurat

## merge assays for multimodel

In [ ]:
merged_seurat <- CreateSeuratObject(counts = flattened_data, project = "NND", min.cells = 5, min.features = 1,assay='FLAT')
# merged_seurat <- merged_seurat[,!colnames(merged_seurat) %in% toRemove]
Assays(merged_seurat)
merged_seurat
dim(merged_seurat@meta.data)

In [ ]:
temporal_assay <- CreateAssayObject(counts = age_reshaped, min.cells = 5, min.features = 1)
temporal_assay

# !colnames(merged_seurat) %in% colnames(temporal_assay)
toRemove <-  colnames(merged_seurat) [!colnames(merged_seurat) %in% colnames(temporal_assay)]
toRemove


In [ ]:
merged_seurat <- merged_seurat[,!colnames(merged_seurat) %in% toRemove]
merged_seurat[["TEMP"]] <- temporal_assay
############# FUNCTION: ADDS METADATA ###############################
merged_seurat <- preprocess_seurat_object(merged_seurat,meta_data)
dim(merged_seurat@meta.data)

In [ ]:
head(merged_seurat@meta.data)

In [ ]:
# meta_data[match(names(merged_seurat$orig.ident), rownames(meta_data)),'FTD']
# meta_data[match(names(merged_seurat$orig.ident), rownames(meta_data)),'FTD']
# seurat_object[['FTD_PRS']] <- meta_data[match(names(seurat_object$orig.ident), rownames(meta_data)),'FTD']
# ad_diagnosis_quality <- ifelse(meta_data$neuropathological_diagnosis == "AD" & meta_data$wrong_diagnosis == 1, 1, 0)
# # merged_seurat[["AD_diagnosis_quality"]] <- ad_diagnosis_quality

In [ ]:
### i cant get the jackstraw to work on the TEMP assay when merged, here ill do it solo first, to get a feel for good parameters
# temp <- CreateSeuratObject(counts = age_reshaped, min.cells = 5, min.features = 1,assay='TEMP')
# all.symptombins <- rownames(temp[["TEMP"]])
# temp <- preprocess_seurat_object(temp,meta_data)
# temp <- NormalizeData(temp,normalization.method = "LogNormalize", scale.factor = 100)
# temp <- ScaleData(temp, features = all.symptombins,do.center = TRUE,scale.max = 10, model.use = "linear")
# temp <- FindVariableFeatures(temp, selection.method = "vst", nfeatures = 1500)
# temp <- perform_pca_and_visualization(temp,rname='pca')
# print(ElbowPlot(temp,reduction='pca'))
# temp <- JackStraw(temp, num.replicate = 100,assay='TEMP')
# temp <- ScoreJackStraw(temp, dims = 1:20)
# print(JackStrawPlot(temp, dims = 1:20))

In [ ]:
DefaultAssay(merged_seurat) <- 'FLAT'
all.symptombins <- rownames(merged_seurat[["FLAT"]])
merged_seurat <- NormalizeData(merged_seurat,normalization.method = "LogNormalize", scale.factor = 100)
merged_seurat <- ScaleData(merged_seurat, features = all.symptombins,do.center = TRUE,scale.max = 10, model.use = "linear")
merged_seurat <- FindVariableFeatures(merged_seurat, selection.method = "vst", nfeatures = 1500)

############ FUNCTION TO DO PCA ##############################
merged_seurat <- perform_pca_and_visualization(merged_seurat,rname='pca')

print(ElbowPlot(merged_seurat,reduction='pca'))
merged_seurat <- JackStraw(merged_seurat, num.replicate = 100,assay='FLAT')
merged_seurat <- ScoreJackStraw(merged_seurat, dims = 1:20)
print(JackStrawPlot(merged_seurat, dims = 1:20))

In [ ]:
DefaultAssay(merged_seurat) <- 'TEMP'
all.symptombins <- rownames(merged_seurat[["TEMP"]])
merged_seurat <- NormalizeData(merged_seurat,normalization.method = "LogNormalize", scale.factor = 100)
merged_seurat <- ScaleData(merged_seurat, features = all.symptombins,do.center = TRUE,scale.max = 10, model.use = "linear")
merged_seurat <- FindVariableFeatures(merged_seurat, selection.method = "vst", nfeatures = 1500)

############ FUNCTION TO DO PCA ##############################
merged_seurat <- perform_pca_and_visualization(merged_seurat,rname='apca')

print(ElbowPlot(merged_seurat,reduction='apca'))

In [ ]:
merged_seurat
merged_seurat[['pca']]@assay.used
merged_seurat[['apca']]@assay.used

In [ ]:
merged_seurat <- FindMultiModalNeighbors(
    merged_seurat,
    k.nn = 20,
    reduction.list = list("pca", "apca"),
    dims.list = list(1:10, 1:30),
    modality.weight.name = list("FLAT.weight","TEMP.weight")
)

## integrates multiple modalities, finds the nearest neighbor based on these
## dims.list: which dimensions to use for each reduction
## k.nn : how many multimodel neighbors. currently at 20
## more: smaller, more distinct clusters, good for fine-grained subpopulations!. robuster. might also break down larger, biologically meaningful clusters
## less: faster, less overfitting (noisy donors become clusters?)
## modality.weight.name Variable name to store modality weight in object meta data

## uf you want to access the multimodal neighbors. info:
# head(merged_seurat@meta.data)
str(merged_seurat[['weighted.nn']])
## nn.idx represents the indices of the nearest neighbors for each donor
## can be accessed with:
#slot(merged_seurat$weighted.nn, "nn.idx")
## nn.dist has the distances of each donor to each nearest neigbor

## acces the WNN graph (weighted)
# merged_seurat[["wknn"]]
## this is the weighted nearest neighbour graph: for each cell, this graph denotes the most similar cells in the dataset
## learns the relative utility of each data type in each cell,
## this is used for the UMAP/clustering. can also be used for the identification of developmental trajectories!

## SNN graph used for clustering (shared)
# merged_seurat[["wsnn"]]

## Donor-specific modality weights can be accessed at 
# merged_seurat$FLAT.weight

In [ ]:
dim(merged_seurat@meta.data)

In [ ]:
merged_seurat <- RunUMAP(merged_seurat,
                         # dims = 1:15,
                         nn.name = "weighted.nn",
                         reduction.name = "wnn.umap",
                         reduction.key = "wnnUMAP_")

## nn.name Name of knn output on which to run UMAP. here we take the info from the neighbors slot
## reduction.name Name to store dimensional reduction under in the Seurat object. in this case umap ON the wnn
## note you know have 3, pca for FLAT, apca for TEMP, wnn.umap for the merged
# str(merged_seurat)
# str(merged_seurat@reductions$wnn.umap)

In [ ]:
merged_seurat <- FindClusters(merged_seurat,
                              resolution = 0.5,
                              graph.name = "wsnn",
                              algorithm = 3,
                              verbose = TRUE)

In [ ]:
# head(merged_seurat@meta.data)

In [ ]:
w = 10
h = 7.5
options(repr.plot.width = w, repr.plot.height = h)
pticksize = 40
p1 <- DimPlot(merged_seurat, reduction = 'wnn.umap', label = FALSE, repel = TRUE, pt.size=2.5, label.size = 10, cols="Dark2",shuffle=TRUE,na.value=2) 
p2 <- DimPlot(merged_seurat, reduction = 'wnn.umap', group.by = 'Age', label = TRUE, repel = TRUE, label.size = 2.5) 
####### simplified fiagnosis: merged diagnoses together into simpler broad categories
p3 <- DimPlot(merged_seurat, reduction = 'wnn.umap', group.by = 'simplified_diagnosis', label = FALSE,
              raster = NULL,shape.by = 'clin_coherence',pt.size=2.5,shuffle=TRUE,seed=3,label.box=FALSE,
              repel = TRUE, label.size = 4,cols=diagnosis_colors) #diagnosis_colors
p1 <- p1  + theme(axis.title.x = element_text(size = pticksize),
            axis.title.y = element_text(size = pticksize),
            # legend.text = element_text(size = pticksize),
                  legend.position = "none",
            axis.text.x = element_text(size = pticksize),
           axis.text.y = element_text(size = pticksize))+ coord_cartesian(xlim = c(-8, 7), ylim = c(-5, 7))
p3 <- p3 + theme(axis.title.x = element_text(size = pticksize),
           axis.title.y = element_text(size = pticksize),
                 # legend.position = "none",
           # legend.text = element_text(size = pticksize),
           axis.text.x = element_text(size = pticksize),
           axis.text.y = element_text(size = pticksize))+ coord_cartesian(xlim = c(-8,7), ylim = c(-5, 7))
p1
p3
# Save p1 to a PDF file
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/main/dimplot_main_cluster.pdf", plot = p1, width = w, height = h,dpi=300)

# Save p3 to a PDF file
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/main/dimplot_main_diagnosis.pdf", plot = p3, width = w, height = h,dpi=300)

In [ ]:
library(ggplot2)
library(ggforce)
unique_diagnoses <- unique(merged_seurat$simplified_diagnosis)
unique_diagnoses
unique_diagnoses <- c('AD','CON','MS','FTD','PD','PSYCH','MND','MSA','PSP','other_dem','DLB','ATAXIA')
options(repr.plot.width = w, repr.plot.height = h)
############### FUNCTION TO MAKE A DIMPLOT WITH CIRCLES AROUND DIAGNOSES OF INTEREST #######
## returns a plot and a list of donorids for each diagnosis that are 'outside' the circle
returns <- generate_dim_plot(merged_seurat, unique_diagnoses,diag='simplified_diagnosis',p3)
returns[[2]]
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/main/dimplot_main_diagnosis.pdf", plot = returns[[2]], width = w, height = h,dpi=300)

# unique_diagnoses <- c('other_dem','VD','DLB','ATAXIA')
# unique_diagnoses <- unique(merged_seurat$simplified_diagnosis)
# returns <- generate_dim_plot(merged_seurat, unique_diagnoses,diag='simplified_diagnosis',p3)          
# returns[[2]]

In [ ]:
rownames(table(merged_seurat@meta.data$simplified_diagnosis))

In [ ]:
library(tidyverse)
options(repr.plot.width = 8, repr.plot.height = 10)
# barplotset <- RenameIdents(object = merged_seurat,"0" = "AD","1" = "PD","2" = "AD/FTD","3" = "CONTROL","4" = "MS", "5" = "PSYCH")
options(repr.plot.width = 12, repr.plot.height = 12)
# p_values_df <- calculate_p_values(merged_seurat,diagnosis_column="simplified_diagnosis")
# corrected_p_values_df <- p_values_df %>%
#   mutate(across(everything(), ~ p.adjust(.x, method = "BH")))
# unique(rownames(corrected_p_values_df))
# plot <- plot_cluster_diagnosis(merged_seurat,corrected_p_values_df,diagnosis_column="simplified_diagnosis")
# head(p_values_df)
# plot
# 'AD','CON','MS','FTD','PD','PSYCH','MND','MSA'
# variables_to_loop <- c("model_and_clinic1","model_and_clinic2","model_and_clinic3","model_and_clinic4", "clin1", "clin2")#,
###### FUNCTION THAT LOOKS FOR OVERREPRESENTATION OF DIAGNOSES
diagnoses <- c('AD','FTD','MS','ATAXIA','DLB','MSA','PD','MND','PSP','CON','VD','PSYCH','other_dem')
diagnoses <- rownames(table(merged_seurat@meta.data$simplified_diagnosis))
print(diagnoses)
known_clin_diag <- c('AD','FTD','MS','ATAXIA','DLB','MSA','PD','MND','PSP','CON','VD')
# diagnoses <- c('MS')
p_values <- calculate_p_values_diag(merged_seurat,diagnosis_column = "simplified_diagnosis",variable_to_loop='clin_coherence',diagnoses,known_clin_diag,verbose=FALSE)
# p_values_df_diag

p_values_diagnoses <- p_values$p_values
p_values_coherence <- p_values$p_values_diag

p_values_diagnoses <- p_values_diagnoses %>%
  spread(key = Cluster, value = P_Value)
rownames(p_values_diagnoses) <- p_values_diagnoses$Diagnosis
p_values_diagnoses <- p_values_diagnoses[, -1]
# print(p_values_diagnoses)
p_values_diagnoses <- p_values_diagnoses %>%
  mutate(across(everything(), ~ p.adjust(.x, method = "BH")))
p_values_diagnoses

p_values_coherence <- p_values_coherence %>%
  spread(key = Cluster, value = P_Value)
rownames(p_values_coherence) <- p_values_coherence$Diagnosis
p_values_coherence <- p_values_coherence[, -1]
# print(p_values_coherence)
p_values_coherence <- p_values_coherence %>%
  mutate(across(everything(), ~ p.adjust(.x, method = "BH")))
p_values_coherence



identitybars <- plot_cluster_diagnosis_diag(merged_seurat,p_values_diagnoses,diagnosis_column = "simplified_diagnosis",color='white',p_values_coherence)     
identitybars <- identitybars + theme(axis.title.x = element_text(size = 24),
           axis.title.y = element_text(size = 24),
           legend.text = element_text(size = 24),
           axis.text.x = element_text(size = 24),
           axis.text.y = element_text(size = 24))#+ coord_cartesian(xlim = c(-6, 6), ylim = c(-6, 6))
identitybars
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/main/identitybars_main_cluster.pdf", plot = identitybars, width = 15, height = 15,dpi=300)



In [ ]:
library(ggsignif)
library(ggpubr)
options(repr.plot.width = 7, repr.plot.height = 6)
mainage <- VlnPlot(
  object = merged_seurat,
  features = 'Age',
  pt.size=0,  
  split.by='sex',
  split.plot=TRUE,
  cols = c('#4682B4', '#FF8C00')  
  # group.by = 'seurat_clusters',
  # cols = viridis(length(unique(merged_cog$seurat_clusters)), option = 'D'),
  )

# VlnPlot(
#   object = merged_seurat,
#   features = 'File',
#   # group.by = 'seurat_clusters',
#   # cols = viridis(length(unique(merged_cog$seurat_clusters)), option = 'D'),
#   pt.size = 0
# )

mainage <- mainage + theme(axis.title.x = element_text(size = 24),
           axis.title.y = element_text(size = 24),
           legend.text = element_text(size = 24),
           axis.text.x = element_text(size = 24),
           axis.text.y = element_text(size = 24)) 
mainage 
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/main/age_main_cluster.pdf", plot = mainage, width = 15, height = 15,dpi=300)


In [ ]:
## save the results of findmarkers (with the old names for now))
library(openxlsx)
options(repr.plot.width = 10, repr.plot.height = 12)

# DefaultAssay(merged_seurat) <- 'FLAT'
extended_cluster_markers_flat = find_extended_cluster_markers(merged_seurat,0.05,method='all',assay='FLAT',verbose=FALSE) 
# filtered_df <- extended_cluster_markers_flat %>%
#   filter(cluster == "3" & p_val_adj < 0.05 &(pct.2-pct.1 <0) & avg_log2FC >0)
# head(filtered_df,50)

DefaultAssay(merged_seurat) <- 'TEMP'
extended_cluster_markers_temp = find_extended_cluster_markers(merged_seurat,0.05,method='all',assay='TEMP',verbose=FALSE) 
# filtered_df <- extended_cluster_markers_temp %>%
#   filter(cluster == "5" & avg_log2FC > 0&p_val_adj < 0.05&(pct.2-pct.1 <0))
# dim(filtered_df)
# filtered_df$marker

# filtered_df <- filtered_df[filtered_df$p_val_adj < 0.05, ]
# filtered_df <- filtered_df %>%
#   mutate(marker = gsub("\\.|\\d", "", marker)) %>%
#   pull(marker)
# unique(filtered_df)

# # ## saving alle markers
file_path <- "/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_analysis/clustermarkers/"
file_name_flat <- "mainclustermarkerflattened.xlsx"
file_name_temp <- "mainclustermarkertemporal.xlsx"
flat_path <- paste0(file_path, file_name_flat)
temp_path <- paste0(file_path, file_name_temp)
write.xlsx(extended_cluster_markers_flat, flat_path, row.names = FALSE)
write.xlsx(extended_cluster_markers_temp, temp_path, row.names = FALSE)

In [ ]:
returns <- create_marker_df(extended_cluster_markers_temp, extended_cluster_markers_flat, sup3)
w=8
h=10
combined_marker_list <- returns[[1]]
combined_marker_list
# # combined_marker_list
annotation_df <- returns[[2]]
# annotation_df
inge_markers <- annotation_df[annotation_df$Domain %in% c("Cognitive", "Psychiatric","General"), ]$ITname
# inge_markers
savepath_main = "/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/main/heatmap_main_cluster.pdf"
returns <- generate_heatmap_plot(merged_seurat,assay='FLAT', combined_marker_list, annotation_df,6,savepath=savepath_main)
mainmarkers <- "/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_analysis/clustermarkers/combined_mainmarkers.txt"
writeLines(returns, con = mainmarkers)
returns

In [ ]:
# head(merged_seurat@meta.data)
cluster_df <- data.frame(DonorID = rownames(merged_seurat@meta.data), seurat_clusters = merged_seurat@meta.data$seurat_clusters)
# head(df2)
seurat_clusters <- merge(df, cluster_df, by = "DonorID", all.x = TRUE)

tail(seurat_clusters)
write.csv(seurat_clusters, file = '/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_analysis/data/seurat_output.csv', row.names = FALSE)


In [ ]:
# head(merged_seurat@meta.data)
colnames(merged_seurat@meta.data)
dim(merged_seurat@meta.data[(merged_seurat@meta.data[['model_and_clinic3']]=='non-coherent' & merged_seurat@meta.data[['simplified_diagnosis']]=='AD'), ])


In [ ]:
print(table(merged_seurat@meta.data$APOE))

apoe <- data.frame(DonorID = rownames(merged_seurat@meta.data),
                    seurat_clusters = merged_seurat@meta.data$seurat_clusters,
                    diagnosis = merged_seurat@meta.data[['simplified_diagnosis']],
                    quality =   merged_seurat@meta.data[['clin_coherence']],
                    apoe = merged_seurat@meta.data[['APOE']]
)    
head(apoe)

apoe2 <- apoe %>%
    group_by(seurat_clusters) %>%
      summarize(
        apoe_44 = sum(!is.na(apoe) & apoe == 44),
        total_donors = sum(!is.na(apoe))
      ) %>%
      mutate(percentage_apoe_44 = (apoe_44 / total_donors) * 100)


apoe2 <- apoe %>%
  filter(diagnosis == "AD") %>%
  group_by(seurat_clusters) %>%
  # filter(seurat_clusters %in% c( 2, 3)) %>%
  summarize(
        apoe_44 = sum(!is.na(apoe) & apoe == 44),
        total_donors = sum(!is.na(apoe))
      ) %>%
  mutate(percentage_apoe_44 = (apoe_44 / total_donors) * 100)


print(apoe2)

clusters <- unique(apoe2$seurat_clusters)
p_values <- data.frame(Cluster = integer(), P_Value = numeric(), EST_OR = numeric())
for (cluster in clusters) {
  sub_df <- apoe2[apoe2$seurat_clusters == cluster, ]
  apoe_44_current <- sub_df$apoe_44
  apoe_44_other <- sum(apoe2$apoe_44) - apoe_44_current
  total_current <- sub_df$total_donors - apoe_44_current
  total_other <- sum(apoe2$total_donors) - total_current
    
  contingency_table <- matrix(c(apoe_44_current, apoe_44_other, total_current, total_other), nrow = 2)
  print(contingency_table)
  test_result <- fisher.test(contingency_table, alternative = "two.sided")
  # test_result <- chisq.test(contingency_table,y=NULL)  
    
  # print(test_result)  
  # print(test_result$estimate)    
  p_values <- rbind(p_values, data.frame(Cluster = cluster, P_Value = test_result$p.value, EST_OR = test_result$estimate))  
}

p_values$bh_corrected <- p.adjust(p_values$P_Value, method = "fdr")
print(p_values) 


In [ ]:
c044 <- apoe2$apoe_44[apoe2$seurat_clusters == 0]
c344 <- apoe2$apoe_44[apoe2$seurat_clusters == 2]
c0t <- apoe2$total_donors[apoe2$seurat_clusters == 0] - c044
c3t <- apoe2$total_donors[apoe2$seurat_clusters == 2] - c344

print(c044)
print(c344)
print(c0t)
print(c3t)
group_A <- c(rep(0, c0t), rep(1, c044)) 
group_B <- c(rep(0, c3t), rep(1, c344))

# Create a 2x2 table
data <- matrix(c(
  sum(group_A == 0), sum(group_A == 1),
  sum(group_B == 0), sum(group_B == 1)
), nrow = 2)

print(data)

# Perform Fisher's exact test
result <- fisher.test(data)

# Print the test result
print(result)

In [ ]:


#######################################################################################################################
# results_list <- list()
# # unique_diagnoses <- c('AD','CON','MS','FTD','PD','PSYCH')
# unique_diagnoses <- c('AD','CON','MS','FTD','PD','MSA','VD','DLB','ATAXIA')#,'MND')
# # unique_diagnoses <- c('AD','FTD')
# variables_to_loop <- c("model_and_clinic1","model_and_clinic2","model_and_clinic3","model_and_clinic4", "clin1", "clin2")#,

# ############### FUNCTION THAT, FOR EACH VARIABLE AND DIAGNOSIS METHOD, SHOWS IF THE CIRCLE SEPARATES COHERENT FROM NON-COHERENT WELL
# combined_results <- coherent_analysis(unique_diagnoses, variables_to_loop, merged_seurat,out_ids)
# combined_results

# result_summary <- combined_results %>%
#   group_by(method) %>%
#   summarize(average_average_effect = mean(average_effect))
# result_summary
# # Print the summary
# print(result_summary)
# options(repr.plot.width = 12, repr.plot.height = 8)
# plot <- ggplot(combined_results, aes(x = diagnosis, y = coh_inside_effect, fill = method)) +
#   geom_bar(stat = "identity", position = "dodge") +
#   labs(title = "Bar Plot of coh_inside_effect by Diagnosis and Method",
#        x = "Diagnosis",
#        y = "coh_inside_effect") +
#   theme_minimal() +
#   theme(axis.text.x = element_text(angle = 45, hjust = 1))
# plot

# plot <- ggplot(combined_results, aes(x = diagnosis, y = non_coh_outside_effect, fill = method)) +
#   geom_bar(stat = "identity", position = "dodge") +
#   labs(title = "Bar Plot of non_coh_outside_effect by Diagnosis and Method",
#        x = "Diagnosis",
#        y = "non_coh_outside_effect") +
#   theme_minimal() +
#   theme(axis.text.x = element_text(angle = 45, hjust = 1))
# plot

# plot <- ggplot(combined_results, aes(x = diagnosis, y = average_effect, fill = method)) +
#   geom_bar(stat = "identity", position = "dodge") +
#   labs(title = "Bar Plot of average_effect by Diagnosis and Method",
#        x = "Diagnosis",
#        y = "average_effect") +
#   theme_minimal() +
#   theme(axis.text.x = element_text(angle = 45, hjust = 1))
# plot


#######################################################################################################################
# p3 <- DimPlot(merged_seurat, reduction = 'wnn.umap', group.by = 'simplified_diagnosis', label = TRUE,
#               raster = NULL, shape.by = 'diagnosis_info', pt.size = 3, shuffle = TRUE, seed = 3,
#               label.box = TRUE, repel = TRUE, label.size = 4, cols = diagnosis_colors)

# # Extract UMAP coordinates and cluster labels
# umap_coords <- as.data.frame(Embeddings(merged_seurat, 'wnn.umap'))
# umap_coords$cluster <- merged_seurat$seurat_clusters
# umap_coords$simplified_diagnosis <- merged_seurat$simplified_diagnosis
# unique_diagnoses <- c('AD','CON','MS','FTD','PD','PSYCH')
# # umap_coords <- umap_coords %>%
# #   filter(simplified_diagnosis %in% unique_diagnoses)
# filtered <- umap_coords %>%
#   filter(
#     (cluster == 0 & simplified_diagnosis == "AD") |
#     (cluster == 1 & simplified_diagnosis == "PD") |
#     (cluster == 2 & simplified_diagnosis == "FTD") |
#     (cluster == 3 & simplified_diagnosis == "CON") |
#     (cluster == 4 & simplified_diagnosis == "MS") |
#     (cluster == 5 & simplified_diagnosis == "PSYCH")
#   )

# # Create UMAP plot with ellipses using ggforce
# p3_with_ellipses <- ggplot(umap_coords, aes(x = wnnUMAP_1, y = wnnUMAP_2, color = factor(simplified_diagnosis))) +
#   geom_point(size = 3) +
#   geom_mark_ellipse(data=filtered,aes(fill = simplified_diagnosis,label=simplified_diagnosis),expand = unit(-5, "mm"),radius = 0) +
#   theme_minimal() +
#   theme(legend.position = "none")

# # Print the UMAP plot with ellipses
# print(p3_with_ellipses)

#######################################################################################################################
## zijn er verschillen in APOE?
# apoe <- data.frame(DonorID = rownames(merged_seurat@meta.data),
#                     seurat_clusters = merged_seurat@meta.data$seurat_clusters,
#                     diagnosis = merged_seurat@meta.data[['simplified_diagnosis']],
#                     quality =   merged_seurat@meta.data[['model_and_clinic3']],
#                      apoe = merged_seurat@meta.data[['APOE']]
# )    
# order_levels <- c("coherent","non-coherent")
# apoe$quality <- factor(apoe$quality, levels = order_levels)
# apoe <- apoe %>%
#   filter(diagnosis == "AD" & seurat_clusters %in% c(2)& !is.na(apoe)) %>%
#   group_by(seurat_clusters, apoe) %>%
#   summarise(count = n()) %>%
#   ungroup() %>%
#   group_by(seurat_clusters) %>%
#   mutate(total_count = sum(count),
#          percentage = (count / total_count) * 100)

# # print(apoe)

# apoe
# data <- matrix(c(36, 261 - 36,13.8, 27, 120 - 27,22.5), ncol = 2)
# rownames(data) <- c("APOE44", "Other Genotypes",'percentage (%)')
# colnames(data) <- c("Cluster 0", "Cluster 2")

# # Perform the chi-squared test
# chisq.test(data)
# data

#######################################################################################################################
# options(repr.plot.width = 10, repr.plot.height = 12)
# sup3_small <- sup3[c("ITname", "Domain", "Grouping")]
# DefaultAssay(merged_seurat) <- 'TEMP'
# extended_cluster_markers = find_extended_cluster_markers(merged_seurat,0.05,method='sign',assay='TEMP',verbose=FALSE) 
# head(extended_cluster_markers)
# file_path <- "/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_analysis/clustermarkers/"
# file_name <- "mainclustermarkertemporal.xlsx"
# full_file_path <- paste0(file_path, file_name)
# write.xlsx(extended_cluster_markers, full_file_path, row.names = FALSE)
# # tail(extended_cluster_markers)
# sub_M1 <- subset(merged_seurat, cells = Cells(merged_seurat), features = extended_cluster_markers$marker)
# DoHeatmap(subset(sub_M1, downsample = 100), features = extended_cluster_markers$marker, size = 4)
# generate_heatmap_plot(merged_seurat,assay='TEMP', extended_cluster_markers$marker, sup3_small,7)

#######################################################################################################################
# # library(ggsignif)
# # library(rstatix)
# # library(dunn.test)
# # library(pairwise)
# ## to do: zoek de verschillen tussen MS-SP
# ad <- data.frame(cluster = Idents(merged_seurat), diagnosis = merged_seurat$simplified_diagnosis,Age=merged_seurat@meta.data$Age)
# # ad <- ad %>%
# #   filter(diagnosis == "AD" )
# # head(ad,10) 
# mean_age_per_cluster <- ad %>%
#   group_by(cluster) %>%
#   summarise(mean_age = mean(Age),std_age = sd(Age),max=max(Age))

# print(mean_age_per_cluster)

# # kruskal_result <- kruskal.test(Age ~ cluster, data = ad)
# # pairwise_dunn <- dunn.test(ad$Age, ad$cluster, method = "bonferroni")
# # print(pairwise_dunn)

# pairwise_kruskal <- pairwise.wilcox.test(ad$Age, ad$cluster, p.adjust.method = "bonferroni")
# # print(pairwise_kruskal$p.value)

# significant_pairs <- data.frame(from = character(), to = character(), p.value = numeric())
# for (i in 1:nrow(pairwise_kruskal$p.value)) {
#   for (j in 1:ncol(pairwise_kruskal$p.value)) {
#     if (!is.na(pairwise_kruskal$p.value[i, j]) && pairwise_kruskal$p.value[i, j] < 0.05) {
#       significant_pairs <- rbind(significant_pairs, data.frame(from = rownames(pairwise_kruskal$p.value)[i],
#                                                                to = colnames(pairwise_kruskal$p.value)[j],
#                                                                p.value = pairwise_kruskal$p.value[i, j]))
#     }
#   }
# }
# significant_pairs$to = as.integer(significant_pairs$to)
# significant_pairs$dist = abs(as.integer(significant_pairs$to) - as.integer(significant_pairs$from))
# significant_pairs <- significant_pairs %>%
#   arrange(desc(dist),to)%>%
#   mutate(height = 100 - (row_number() -1) * 5)
# # significant_pairs$p.star[significant_pairs$p.value < 0.05] <- "*"
# # significant_pairs$p.star[significant_pairs$p.value < 0.005] <- "**"
# significant_pairs$p.star[significant_pairs$p.value < 0.0005] <- "***"
# significant_pairs#$to

# options(repr.plot.width = 7, repr.plot.height = 6)
# # my_comparisons <- list( c(5, 6), c(1,2),c(3,5) )
# g <- VlnPlot(
#   object = merged_seurat,
#   features = 'Age',
#   pt.size=0,  
#   split.by='sex',
#     # y.min = 0,
#   split.plot=TRUE
#   # group.by = 'seurat_clusters',
#   # cols = viridis(length(unique(merged_cog$seurat_clusters)), option = 'D'),
#   )+ scale_y_continuous(limits = c(0,200))+ geom_signif(
#     y_position = significant_pairs$height, xmin = significant_pairs$to+1, xmax = significant_pairs$from,
#     annotation = significant_pairs$p.star, tip_length = 0.02
#   )
# g


#######################################################################################################################
## kleurtjes voor in python code
# library(scales)

# #extract hex color codes for a plot with three elements in ggplot2 
# hex <- hue_pal()(3)
# hex

### AD/FTD cluster

In [ ]:
merged_cog <- subset(merged_seurat, idents = c(0,2))
Assays(merged_cog)

In [ ]:
# seurat_object[['simplified_diagnosis']] <- meta_data[match(names(seurat_object$orig.ident), rownames(meta_data)),'simplified_diagnosis']
merged_cog$cog_cluster_diagnosis <- merged_cog$diagnosis

merged_cog$cog_cluster_diagnosis <- gsub("ATAXIA,FA", "ATAXIA", merged_cog$cog_cluster_diagnosis)
merged_cog$cog_cluster_diagnosis <- gsub("ATAXIA,SCA", "ATAXIA", merged_cog$cog_cluster_diagnosis)
merged_cog$cog_cluster_diagnosis <- gsub("ATAXIA,FXTAS", "ATAXIA", merged_cog$cog_cluster_diagnosis)
merged_cog$cog_cluster_diagnosis <- gsub("PSYCH,SCZ", "PSYCH", merged_cog$cog_cluster_diagnosis)
merged_cog$cog_cluster_diagnosis <- gsub("PSYCH,BP", "PSYCH", merged_cog$cog_cluster_diagnosis)
merged_cog$cog_cluster_diagnosis <- gsub("PSYCH,MDD", "PSYCH", merged_cog$cog_cluster_diagnosis)
merged_cog$cog_cluster_diagnosis <- gsub("MS,MS-PP", "MS", merged_cog$cog_cluster_diagnosis)
merged_cog$cog_cluster_diagnosis <- gsub("MS,MS-SP", "MS", merged_cog$cog_cluster_diagnosis)
merged_cog$cog_cluster_diagnosis <- gsub("MS_undefined", "MS", merged_cog$cog_cluster_diagnosis)

merged_cog$cog_cluster_diagnosis <- gsub("DEM,SICC,AGD", "DEM,SICC", merged_cog$cog_cluster_diagnosis)
merged_cog$cog_cluster_diagnosis <- gsub("FTD,FTD-TDP_undefined", "FTD-TDP", merged_cog$cog_cluster_diagnosis)
merged_cog$cog_cluster_diagnosis <- gsub("FTD,FTD-TDP-C", "FTD-TDP", merged_cog$cog_cluster_diagnosis)
merged_cog$cog_cluster_diagnosis <- gsub("FTD,FTD-TDP-A,PROG", "FTD-TDP", merged_cog$cog_cluster_diagnosis)
merged_cog$cog_cluster_diagnosis <- gsub("FTD,FTD-TDP-B,C9ORF72", "FTD-TDP", merged_cog$cog_cluster_diagnosis)


table(merged_cog[['cog_cluster_diagnosis']])
table(merged_cog[['simplified_diagnosis']])
# ms_cluster[['ms_cluster_diagnosis']]

In [ ]:
DefaultAssay(merged_cog) <- 'FLAT'
all.symptombins <- rownames(merged_cog[["FLAT"]])
merged_cog <- NormalizeData(merged_cog,normalization.method = "LogNormalize", scale.factor = 100)
merged_cog <- ScaleData(merged_cog, features = all.symptombins,do.center = TRUE,scale.max = 10, model.use = "linear")
merged_cog <- FindVariableFeatures(merged_cog, selection.method = "vst", nfeatures = 1500)
merged_cog <- perform_pca_and_visualization(merged_cog,rname='pca')
print(ElbowPlot(merged_cog,reduction='pca'))
merged_cog <- JackStraw(merged_cog, num.replicate = 100,assay='FLAT')
merged_cog <- ScoreJackStraw(merged_cog, dims = 1:20)
print(JackStrawPlot(merged_cog, dims = 1:20))

In [ ]:
DefaultAssay(merged_cog) <- 'TEMP'
all.symptombins <- rownames(merged_cog[["TEMP"]])
merged_cog <- NormalizeData(merged_cog,normalization.method = "LogNormalize", scale.factor = 100)
merged_cog <- ScaleData(merged_cog, features = all.symptombins,do.center = TRUE,scale.max = 10, model.use = "linear")
merged_cog <- FindVariableFeatures(merged_cog, selection.method = "vst", nfeatures = 1500)
merged_cog <- perform_pca_and_visualization(merged_cog,rname='apca') ## change to pca for plots
# print(ElbowPlot(merged_cog,reduction='pca')) ## change to pcs for plots
# merged_cog <- JackStraw(merged_cog, num.replicate = 100,assay='TEMP')
# merged_cog <- ScoreJackStraw(merged_cog, dims = 1:20)
# print(JackStrawPlot(merged_cog, dims = 1:20))

In [ ]:
merged_cog <- FindMultiModalNeighbors(
    merged_cog,
    reduction.list = list("pca", "apca"),
    # dims.list = list(1:5, 1:20), modality.weight.name = list("FLAT.weight","TEMP.weight")
    dims.list = list(1:7, 1:10), modality.weight.name = list("FLAT.weight","TEMP.weight")
)

In [ ]:
merged_cog <- RunUMAP(merged_cog,
                      # dims = 1:10,
                      nn.name = "weighted.nn",
                      reduction.name = "wnn.umap",
                      reduction.key = "wnnUMAP_",
                      verbose=FALSE  
)


In [ ]:
merged_cog <- FindClusters(merged_cog,
                              resolution = 0.4,
                              graph.name = "wsnn",
                              algorithm = 3,
                              verbose = TRUE)

In [ ]:
w = 12
h = 7.5
pticksize = 40
options(repr.plot.width = w, repr.plot.height = h)
p1 <- DimPlot(merged_cog, reduction = 'wnn.umap', label = FALSE,shape.by = 'clin_coherence',
              repel = TRUE, label.size = 5,pt.size=4,cols="Dark2",shuffle=TRUE) +
            theme(axis.title.x = element_text(size = pticksize),
                axis.title.y = element_text(size = pticksize),
                legend.text = element_text(size = pticksize),
                axis.text.x = element_text(size = pticksize),
          # legend.position = "none",
               axis.text.y = element_text(size = pticksize))#+ #coord_cartesian(xlim = c(-3, 3.5), ylim = c(-3, 3))+
    # scale_x_continuous(breaks = c(-3,-2,-1, 0,1,2, 3)) + # Add x-axis tick marks at -3, 0, and 3+
    # scale_y_continuous(breaks = c(-3,-2,-1, 0,1,2, 3))  # Add x-axis tick marks at -3, 0, and 3


# p2 <- DimPlot(psych_cluster, reduction = 'wnn.umap', group.by = 'ms_cluster_diagnosis', label = TRUE, repel = TRUE, label.size = 2.5,cols=diagnosis_colors) 
p3 <- DimPlot(merged_cog, reduction = 'wnn.umap', group.by = 'cog_cluster_diagnosis', label = FALSE, #cog_cluster_diagnosis
              raster = NULL,pt.size=5,shuffle=TRUE,seed=3,label.box=FALSE, shape.by = 'clin_coherence',
              repel = TRUE, label.size = 4,cols=diagnosis_colors) +
    theme(axis.title.x = element_text(size = pticksize),
               axis.title.y = element_text(size = pticksize),
               legend.text = element_text(size = pticksize),
          legend.position = "none",
               axis.text.x = element_text(size = pticksize),
               axis.text.y = element_text(size = pticksize))+ #coord_cartesian(xlim = c(-3, 3.5), ylim = c(-3, 3))+
    # scale_x_continuous(breaks = c(-3,-2,-1, 0,1,2, 3)) + # Add x-axis tick marks at -3, 0, and 3+
    # scale_y_continuous(breaks = c(-3,-2,-1, 0,1,2, 3)) + # Add x-axis tick marks at -3, 0, and 3
    labs(title = "")  # Remove plot title
p1
# w = 20
# h = 12
# options(repr.plot.width = w, repr.plot.height = h)
p3

# Save p1 to a PDF file
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/cog/dimplot_cog_cluster.pdf", plot = p1, width = w, height = h,dpi=300)

# Save p3 to a PDF file
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/cog/dimplot_cog_diagnosis.pdf", plot = p3, width = w, height = h,dpi=300)

In [ ]:

## add circle
w = 20
h = 12
unique_diagnoses <- table(merged_cog$cog_cluster_diagnosis) #cog_cluster_diagnosis
unique_diagnoses
diagnoses <- names(unique_diagnoses[unique_diagnoses > 30])
# diagnoses <- c('MS','CON','DLB','other_dem','VD','DEM,ENCEPHA,VE','AD,ENCEPHA,VE','DLB,SICC','FTD,FTD-TAU,TAU')
diagnoses <- c('FTD-TDP','AD','AD,DLB') #'FTD,PID','FTD-TDP'
diagnoses
options(repr.plot.width = w, repr.plot.height = h)
# ############### FUNCTION TO MAKE A DIMPLOT WITH CIRCLES AROUND DIAGNOSES OF INTEREST #######
# ## returns a plot and a list of donorids for each diagnosis that are 'outside' the circle
returns <- generate_dim_plot(merged_cog, diagnoses,diag='cog_cluster_diagnosis',p3) #cog_cluster_diagnosis
returns[[2]]
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/cog/dimplot_cog_diagnosis.pdf", plot = returns[[2]], width = w, height = h,dpi=300)

##### plot on original cluster

In [ ]:
# copy_of_merged <- merged_seurat
# # copy_of_merged
# # colnames(copy_of_merged@meta.data)
# # colnames(merged_cog@meta.data)
# original_rownames <- colnames(copy_of_merged@meta.data)
# copy_of_merged@meta.data <- merge(copy_of_merged@meta.data, merged_cog@meta.data[, c("seurat_clusters","cog_cluster_diagnosis","simplified_diagnosis"), drop = FALSE], by = "row.names", all.x = TRUE)
# colnames(copy_of_merged@meta.data)[colnames(copy_of_merged@meta.data) == "seurat_clusters.y"] <- "cog_clusters"
# colnames(copy_of_merged@meta.data)[colnames(copy_of_merged@meta.data) == "seurat_clusters.x"] <- "seurat_clusters"
# colnames(copy_of_merged@meta.data)[colnames(copy_of_merged@meta.data) == "simplified_diagnosis.y"] <- "cog_simplified"
# colnames(copy_of_merged@meta.data)[colnames(copy_of_merged@meta.data) == "simplified_diagnosis.x"] <- "simplified_diagnosis"
# rownames(copy_of_merged@meta.data) <- copy_of_merged@meta.data$Row.names
# copy_of_merged@meta.data$Row.names <- NULL
# # original_rownames
# # head(copy_of_merged@meta.data)
# head(copy_of_merged@meta.data[copy_of_merged@meta.data$seurat_clusters == 5, ])

In [ ]:
w = 10
h = 7.5
options(repr.plot.width = w, repr.plot.height = h)
pticksize = 40
cogcells <- WhichCells(merged_seurat, idents = c( "0","2"))
cog_highlight <- DimPlot(merged_seurat, reduction = 'wnn.umap',label = FALSE,cells.highlight = cogcells,cols.highlight='black',sizes.highlight=2,
              raster = NULL,pt.size=1.5,shuffle=TRUE,seed=3,label.box=FALSE,
              repel = TRUE, label.size = 4)

cog_highlight
# Save to a PDF file
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/cog/highlight.pdf", plot = cog_highlight, width = w, height = h,dpi=300)

# # Save p3 to a PDF file
# ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/main/dimplot_main_diagnosis.pdf", plot = p3, width = w, height = h,dpi=300)

##### identity bars
for cog we do the coherence analysis

In [ ]:
w = 20
h = 7
# options(repr.plot.width = w, repr.plot.height = h)
# p_values_df <- calculate_p_values(merged_cog,diagnosis_column = "cog_cluster_diagnosis") #cog_cluster_diagnosis simplified_diagnosis
# # p_values_df
# unique(rownames(p_values_df))
# corrected_p_values_df <- p_values_df %>%
#   mutate(across(everything(), ~ p.adjust(.x, method = "BH")))
# p_values_df
# corrected_p_values_df


# # diagnoses <- c('AD','FTD','MS','ATAXIA','DLB','MSA','PD','MND','PSP','CON','VD')
# # diagnoses <- c('MS')
# diagnoses <- unique(rownames(p_values_df))
# # diagnoses <- names(unique_diagnoses[unique_diagnoses > 10])
# diagnoses
# p_values_df_diag <- calculate_p_values_diag(merged_cog,diagnosis_column = "cog_cluster_diagnosis",
#                                             variable_to_loop='clin_coherence',diagnoses,verbose=FALSE)

# # p_values_df_diag
# corrected_p_values_diag <- p_values_df_diag %>%
#   mutate(across(everything(), ~ p.adjust(.x, method = "BH")))
# # corrected_p_values_diag

# # print(corrected_p_values_df)
# identitybars <- plot_cluster_diagnosis_diag(merged_cog,corrected_p_values_df,
#                                             diagnosis_column = "cog_cluster_diagnosis",color='white',corrected_p_values_diag)     
# identitybars <- identitybars + theme(axis.title.x = element_text(size = 24),
#            axis.title.y = element_text(size = 24),
#            legend.text = element_text(size = 24),
#            axis.text.x = element_text(size = 24),
#            axis.text.y = element_text(size = 24))#+ coord_cartesian(xlim = c(-6, 6), ylim = c(-6, 6))
# identitybars
# ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/cog/identitybars_cog_cluster.pdf",
#        plot = identitybars, width = 15, height = 15,dpi=300)

# library(tidyverse)
options(repr.plot.width = 12, repr.plot.height = 12)
# diagnoses <- c('AD','FTD','MS','ATAXIA','DLB','MSA','PD','MND','PSP','CON','VD','PSYCH')
table(merged_cog@meta.data$cog_cluster_diagnosis)
diagnoses <- rownames(table(merged_cog@meta.data$cog_cluster_diagnosis))
print(list(diagnoses))
known_clin_diag <-  c('AD',"AD-DLB","DLB","FTD-TDP","FTD,FTD-TAU,TAU","FTD,PID","PD","PSP","VD")
print(known_clin_diag)
# diagnoses <- c('MS')
p_values <- calculate_p_values_diag(merged_cog,diagnosis_column = "cog_cluster_diagnosis",variable_to_loop='clin_coherence',diagnoses, known_clin_diag, verbose=FALSE)
# p_values_df_diag

p_values_diagnoses <- p_values$p_values
p_values_coherence <- p_values$p_values_diag

p_values_diagnoses <- p_values_diagnoses %>%
  spread(key = Cluster, value = P_Value)
rownames(p_values_diagnoses) <- p_values_diagnoses$Diagnosis
p_values_diagnoses <- p_values_diagnoses[, -1]
print(p_values_diagnoses)
p_values_diagnoses <- p_values_diagnoses %>%
  mutate(across(everything(), ~ p.adjust(.x, method = "BH")))
p_values_diagnoses

p_values_coherence <- p_values_coherence %>%
  spread(key = Cluster, value = P_Value)
rownames(p_values_coherence) <- p_values_coherence$Diagnosis
p_values_coherence <- p_values_coherence[, -1]
# print(p_values_coherence)
p_values_coherence <- p_values_coherence %>%
  mutate(across(everything(), ~ p.adjust(.x, method = "BH")))
p_values_coherence



identitybars <- plot_cluster_diagnosis_diag(merged_cog,p_values_diagnoses,diagnosis_column = "cog_cluster_diagnosis",color='white',p_values_coherence)     
identitybars <- identitybars + theme(axis.title.x = element_text(size = 24),
           axis.title.y = element_text(size = 24),
           legend.text = element_text(size = 24),
           axis.text.x = element_text(size = 24),
           axis.text.y = element_text(size = 24))#+ coord_cartesian(xlim = c(-6, 6), ylim = c(-6, 6))
identitybars
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/cog/identitybars_cog_cluster.pdf", plot = identitybars, width = w, height = h,dpi=300)



##### heatmap

In [ ]:
DefaultAssay(merged_cog) <- 'FLAT'
extended_cluster_markers_flat = find_extended_cluster_markers(merged_cog,0.05,method='all',assay='FLAT',verbose=FALSE) 

DefaultAssay(merged_cog) <- 'TEMP'
extended_cluster_markers_temp = find_extended_cluster_markers(merged_cog,0.05,method='all',assay='TEMP',verbose=FALSE) 

file_path <- "/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_analysis/clustermarkers/"
file_name_flat <- "cogclustermarkerflattened.xlsx"
file_name_temp <- "cogclustermarkertemporal.xlsx"
flat_path <- paste0(file_path, file_name_flat)
temp_path <- paste0(file_path, file_name_temp)
write.xlsx(extended_cluster_markers_flat, flat_path, row.names = FALSE)
write.xlsx(extended_cluster_markers_temp, temp_path, row.names = FALSE)


In [ ]:
returns <- create_marker_df(extended_cluster_markers_temp, extended_cluster_markers_flat, sup3)
w=8
h=10
combined_marker_list <- returns[[1]]
# combined_marker_list
# # combined_marker_list
annotation_df <- returns[[2]]
# annotation_df
inge_markers <- annotation_df[annotation_df$Domain %in% c("Cognitive", "Psychiatric","General"), ]$ITname
# inge_markers
savepath_main = "/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/cog/heatmap_cog_cluster.pdf"
returns <- generate_heatmap_plot(merged_cog,assay='FLAT', combined_marker_list, annotation_df,9,savepath=savepath_main)
mainmarkers <- "/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_analysis/clustermarkers/combined_cogmarkers.txt"
writeLines(returns, con = mainmarkers)
returns

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 6)
VlnPlot(
  object = merged_cog,
  features = 'Age',
  pt.size=0,  
  split.by='sex',
  split.plot=TRUE
  # group.by = 'seurat_clusters',
  # cols = viridis(length(unique(merged_cog$seurat_clusters)), option = 'D'),
)

# options(repr.plot.width = 7, repr.plot.height = 6)
# VlnPlot(
#   object = merged_cog,
#   features = 'File',
#   pt.size=0,  
#   split.by='sex',
#   split.plot=TRUE
#   # group.by = 'seurat_clusters',
#   # cols = viridis(length(unique(merged_cog$seurat_clusters)), option = 'D'),
# )


In [ ]:
print(table(merged_cog@meta.data$APOE))

apoe <- data.frame(DonorID = rownames(merged_cog@meta.data),
                    seurat_clusters = merged_cog@meta.data$seurat_clusters,
                    diagnosis = merged_cog@meta.data[['simplified_diagnosis']],
                    quality =   merged_cog@meta.data[['clin_coherence']],
                    apoe = merged_cog@meta.data[['APOE']]
)    
head(apoe)

apoe2 <- apoe %>%
    group_by(seurat_clusters) %>%
      summarize(
        apoe_44 = sum(!is.na(apoe) & apoe == 44),
        total_donors = sum(!is.na(apoe))
      ) %>%
      mutate(percentage_apoe_44 = (apoe_44 / total_donors) * 100)


apoe2 <- apoe %>%
  filter(diagnosis == "AD") %>%
  group_by(seurat_clusters) %>%
  summarize(
        apoe_44 = sum(!is.na(apoe) & apoe == 44),
        total_donors = sum(!is.na(apoe))
      ) %>%
  mutate(percentage_apoe_44 = (apoe_44 / total_donors) * 100)


print(apoe2)

clusters <- unique(apoe2$seurat_clusters)
p_values <- data.frame(Cluster = integer(), P_Value = numeric(), EST_OR = numeric())
for (cluster in clusters) {
  sub_df <- apoe2[apoe2$seurat_clusters == cluster, ]
  apoe_44_current <- sub_df$apoe_44
  apoe_44_other <- sum(apoe2$apoe_44) - apoe_44_current
  total_current <- sub_df$total_donors - apoe_44_current
  total_other <- sum(apoe2$total_donors) - total_current
    
  contingency_table <- matrix(c(apoe_44_current, apoe_44_other, total_current, total_other), nrow = 2)
  print(contingency_table)
  test_result <- fisher.test(contingency_table, alternative = "two.sided")
  # test_result <- chisq.test(contingency_table,y=NULL)  
    
  # print(test_result)  
  # print(test_result$estimate)    
  p_values <- rbind(p_values, data.frame(Cluster = cluster, P_Value = test_result$p.value, EST_OR = test_result$estimate))  
}

p_values$bh_corrected <- p.adjust(p_values$P_Value, method = "fdr")
print(p_values) 


In [ ]:
c044 <- apoe2$apoe_44[apoe2$seurat_clusters == 0]
c344 <- apoe2$apoe_44[apoe2$seurat_clusters == 1]
c0t <- apoe2$total_donors[apoe2$seurat_clusters == 0] - c044
c3t <- apoe2$total_donors[apoe2$seurat_clusters == 1] - c344

print(c044)
print(c344)
print(c0t)
print(c3t)
group_A <- c(rep(0, c0t), rep(1, c044)) 
group_B <- c(rep(0, c3t), rep(1, c344))

# Create a 2x2 table
data <- matrix(c(
  sum(group_A == 0), sum(group_A == 1),
  sum(group_B == 0), sum(group_B == 1)
), nrow = 2)

print(data)

# Perform Fisher's exact test
result <- fisher.test(data)

# Print the test result
print(result)

In [ ]:
# library(ggplot2)
# library(ggforce)
# p3 <- DimPlot(merged_cog, reduction = 'wnn.umap', group.by = 'cog_cluster_diagnosis', label = FALSE,
#               raster = NULL,shape.by = 'diagnosis_info',pt.size=5,shuffle=TRUE,seed=3,label.box=TRUE,
#               repel = TRUE, label.size = 4,cols=diagnosis_colors) +
#   theme(
#     legend.text = element_text(size = 20),   # Adjust legend font size
#     legend.title = element_text(size = 16),  # Adjust legend title font size
#     axis.title = element_text(size = 16),    # Adjust axis title font size
#     axis.text = element_text(size = 16)      # Adjust axis tick label font size
#   )

# unique_diagnoses <- unique(merged_cog$simplified_diagnosis)
# unique_diagnoses <- unique(merged_cog$cog_cluster_diagnosis)
# unique_diagnoses
# unique_diagnoses <- c('AD','DEM,SICC',
#                       'CBD','FTD',
#                       # 'AD,ENCEPHA,VE','ATAXIA','CON','DEM,ENCEPHA,VE',
#                       'DLB','PSYCH'#'PD','PD,AD','DLB,SICC'
#                      )
# returns <- generate_dim_plot(merged_cog, unique_diagnoses,diag='simplified_diagnosis',p3)
# out_ids <- returns[[1]]            
# returns[[2]]

# # unique_diagnoses <- c('other_dem','VD','DLB','ATAXIA')
# # unique_diagnoses <- unique(merged_seurat$simplified_diagnosis)
# # returns <- generate_dim_plot(merged_seurat, unique_diagnoses)
# # out_ids <- returns[[1]]            
# # returns[[2]]

######################################################################################################################
# results_list <- list()
# # unique_diagnoses <- c('AD','CON','MS','FTD','PD','PSYCH')
# unique_diagnoses <- c('AD','DEM,SICC',
#                       'CBD','FTD',
#                       'AD,ENCEPHA,VE','ATAXIA','CON','DEM,ENCEPHA,VE',
#                       'DLB','PSYCH','PD','PD,AD','DLB,SICC'
#                      )
# # unique_diagnoses <- c('AD','FTD')
# variables_to_loop <- c("model_and_clinic1","model_and_clinic2", "clin1", "clin2")#, 
# combined_results <- coherent_analysis(unique_diagnoses, variables_to_loop, merged_cog,out_ids)
# # combined_results

# result_summary <- combined_results %>%
#   group_by(method) %>%
#   summarize(average_average_effect = mean(average_effect))
# result_summary
# # Print the summary
# print(result_summary)
# options(repr.plot.width = 12, repr.plot.height = 8)
# plot <- ggplot(combined_results, aes(x = diagnosis, y = coh_inside_effect, fill = method)) +
#   geom_bar(stat = "identity", position = "dodge") +
#   labs(title = "Bar Plot of coh_inside_effect by Diagnosis and Method",
#        x = "Diagnosis",
#        y = "coh_inside_effect") +
#   theme_minimal() +
#   theme(axis.text.x = element_text(angle = 45, hjust = 1))
# plot

# plot <- ggplot(combined_results, aes(x = diagnosis, y = non_coh_outside_effect, fill = method)) +
#   geom_bar(stat = "identity", position = "dodge") +
#   labs(title = "Bar Plot of non_coh_outside_effect by Diagnosis and Method",
#        x = "Diagnosis",
#        y = "non_coh_outside_effect") +
#   theme_minimal() +
#   theme(axis.text.x = element_text(angle = 45, hjust = 1))
# plot

# plot <- ggplot(combined_results, aes(x = diagnosis, y = average_effect, fill = method)) +
#   geom_bar(stat = "identity", position = "dodge") +
#   labs(title = "Bar Plot of average_effect by Diagnosis and Method",
#        x = "Diagnosis",
#        y = "average_effect") +
#   theme_minimal() +
#   theme(axis.text.x = element_text(angle = 45, hjust = 1))
# plot

######################################################################################################################
# options(repr.plot.width = 15, repr.plot.height = 15)

# library(ggplot2)
# library(ggforce)
# unique_diagnoses <- unique(merged_cog$simplified_diagnosis)
# unique_diagnoses
# unique_diagnoses <- c('AD','other_dem','FTD')
# plot <- generate_dim_plot(merged_cog,unique_diagnoses)
# plot

######################################################################################################################
# options(repr.plot.width = 12, repr.plot.height = 8)
# p_values_df <- calculate_p_values(merged_cog,diagnosis_column="cog_cluster_diagnosis")
# corrected_p_values_df <- p_values_df %>%
#   mutate(across(everything(), ~ p.adjust(.x, method = "BH")))
# corrected_p_values_df
# plot <- plot_cluster_diagnosis(merged_cog,corrected_p_values_df,diagnosis_column="cog_cluster_diagnosis")

# plot

# ###### FUNCTION THAT LOOKS FOR OVERREPRESENTATION OF DIAGNOSES


######################################################################################################################
# options(repr.plot.width = 12, repr.plot.height = 6)

# variables_to_loop <- c("model_and_clinic1","model_and_clinic2","model_and_clinic3","model_and_clinic4", "clin1", "clin2")#,
# ###### FUNCTION THAT LOOKS FOR OVERREPRESENTATION OF DIAGNOSES
# # head(merged_cog@meta.data)
# diagnoses <- c('AD','DEM,SICC','FTD_undefined','VD','FTD,FTD-TDP_undefined','AD,DLB',
#                'DLB,SICC','DEM,SICC,AGD','FTD,FTD-TDP-B,C9ORF72','DEM,ENCEPHA,VE','PSYCH',
#                'AD,CA','CBD','FTD,FTD-TDP-A,PROG','MS','FTD,PID','AD,ENCEPHA,VE','FTD,FTD-TAU,TAU',
#                'PSP','PD','CON','DLB','FTD,FTD-TDP,MND','ATAXIA','FTD,FTD-FUS','FTD,FTD-TDP-C','PD,AD','MSA','MS_undefined')
# ## for simplified
# # diagnoses <- c('AD','DLB','FTD')
# ## for uitgesplitst
# diagnoses <- c('AD','DLB','FTD,FTD-TDP_undefined','AD,DLB','FTD,FTD-TDP-B,C9ORF72','FTD,PID')
# p_values_df_diag <- calculate_p_values_diag(merged_cog,diagnosis_column = "cog_cluster_diagnosis",variable_to_loop='model_and_clinic3',diagnoses,verbose=FALSE)
# p_values_df_diag
# corrected_p_values_diag <- p_values_df_diag %>%
#   mutate(across(everything(), ~ p.adjust(.x, method = "BH")))
# corrected_p_values_diag
# # # print(corrected_p_values_df)
# plot <- plot_cluster_diagnosis_diag(merged_cog,corrected_p_values_df,diagnosis_column = "cog_cluster_diagnosis",color='white',corrected_p_values_diag)     
# plot



######################################################################################################################


######################################################################################################################
# options(repr.plot.width = 7, repr.plot.height = 6)
# VlnPlot(
#   object = merged_cog,
#   features = 'Age',
#   pt.size=0,  
#   split.by='sex',
#   split.plot=TRUE
#   # group.by = 'seurat_clusters',
#   # cols = viridis(length(unique(merged_cog$seurat_clusters)), option = 'D'),
# )
# # VlnPlot(
# #   object = merged_cog,
# #   features = 'File',
# #   # group.by = 'seurat_clusters',
# #   # cols = viridis(length(unique(merged_cog$seurat_clusters)), option = 'D'),
# #   pt.size = 0
# # )

# ad <- data.frame(cluster = Idents(merged_cog), Age=merged_cog@meta.data$Age)
# mean_age_per_cluster <- ad %>%
#   group_by(cluster) %>%
#   summarise(mean_age = mean(Age),std_age = sd(Age),max=max(Age))
# pairwise_kruskal <- pairwise.wilcox.test(ad$Age, ad$cluster, p.adjust.method = "bonferroni")
# significant_pairs <- data.frame(from = character(), to = character(), p.value = numeric())
# print(mean_age_per_cluster)
# for (i in 1:nrow(pairwise_kruskal$p.value)) {
#   for (j in 1:ncol(pairwise_kruskal$p.value)) {
#     if (!is.na(pairwise_kruskal$p.value[i, j]) && pairwise_kruskal$p.value[i, j] >= 0.05) {
#       significant_pairs <- rbind(significant_pairs, data.frame(from = rownames(pairwise_kruskal$p.value)[i],
#                                                                to = colnames(pairwise_kruskal$p.value)[j],
#                                                                p.value = pairwise_kruskal$p.value[i, j]))
#     }
#   }
# }
# significant_pairs$to = as.integer(significant_pairs$to)
# significant_pairs$dist = abs(as.integer(significant_pairs$to) - as.integer(significant_pairs$from))
# significant_pairs <- significant_pairs %>%
#   arrange(desc(dist),to)%>%
#   mutate(height = 150 - (row_number() -1) * 5)
# significant_pairs$p.star[significant_pairs$p.value < 0.05] <- "*"
# significant_pairs$p.star[significant_pairs$p.value < 0.005] <- "**"
# significant_pairs$p.star[significant_pairs$p.value < 0.0005] <- "***"
# significant_pairs#$to
# g <- VlnPlot(
#   object = merged_cog,
#   features = 'Age',
#   pt.size=0,  
#   split.by='sex',
#   split.plot=TRUE
#   )+ scale_y_continuous(limits = c(0,150))+ geom_signif(
#     y_position = significant_pairs$height, xmin = significant_pairs$to+1, xmax = significant_pairs$from,
#     annotation = 'NS', tip_length = 0.02
#   )
# g

######################################################################################################################
# ## to do: zoek de verschillen tussen MS-SP
# ad <- data.frame(cluster = Idents(merged_cog), diagnosis = merged_cog$simplified_diagnosis,Age=merged_cog@meta.data$Age)
# ad <- ad %>%
#   filter(diagnosis == "AD" )
# head(ad) 
# mean_age_per_cluster <- ad %>%
#   group_by(cluster) %>%
#   summarise(mean_age = mean(Age),std_age = sd(Age))

# print(mean_age_per_cluster)
# cluster_1 <- subset(ad, cluster == "0")
# cluster_2 <- subset(ad, cluster == "1")

# # Perform Mann-Whitney U test
# mwu_result <- wilcox.test(cluster_1$Age, cluster_2$Age)

# # Summarize Mann-Whitney U test results
# print(mwu_result)

# ggplot(ad, aes(x = cluster, y = Age, fill = cluster)) +
#   geom_violin() +
#   labs(x = "Cluster", y = "Age") + theme_minimal()



In [ ]:
# head(merged_seurat@meta.data)
cog_df <- data.frame(DonorID = rownames(merged_cog@meta.data), cog_clusters = merged_cog@meta.data$seurat_clusters)
# head(df2)
# seurat_clusters <- merge(df, cluster_df, by = "DonorID", all.x = TRUE)
cogmain <- merge(seurat_clusters, cog_df, by = "DonorID", all.x = TRUE)
tail(cogmain)
write.csv(cogmain, file = '/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_analysis/data/seurat_output.csv', row.names = FALSE)


### MS cluster

In [ ]:
ms_cluster <- subset(merged_seurat, idents = c(4))
Assays(ms_cluster)


In [ ]:
# seurat_object[['simplified_diagnosis']] <- meta_data[match(names(seurat_object$orig.ident), rownames(meta_data)),'simplified_diagnosis']
ms_cluster$ms_cluster_diagnosis <- ms_cluster$diagnosis

ms_cluster$ms_cluster_diagnosis <- gsub("ATAXIA,FA", "ATAXIA", ms_cluster$ms_cluster_diagnosis)
ms_cluster$ms_cluster_diagnosis <- gsub("ATAXIA,SCA", "ATAXIA", ms_cluster$ms_cluster_diagnosis)
ms_cluster$ms_cluster_diagnosis <- gsub("ATAXIA,ADCA", "ATAXIA", ms_cluster$ms_cluster_diagnosis)
ms_cluster$ms_cluster_diagnosis <- gsub("PSYCH,MDD", "PSYCH", ms_cluster$ms_cluster_diagnosis)
ms_cluster$ms_cluster_diagnosis <- gsub("FTD,FTD-TDP-B,C9ORF72", "FTD", ms_cluster$ms_cluster_diagnosis)
ms_cluster$ms_cluster_diagnosis <- gsub("CBD", "other_dem", ms_cluster$ms_cluster_diagnosis)
ms_cluster$ms_cluster_diagnosis <- gsub("MND,ALS", "ALS", ms_cluster$ms_cluster_diagnosis)
ms_cluster$ms_cluster_diagnosis <- gsub("DEM,SICC", "other_dem", ms_cluster$ms_cluster_diagnosis)
ms_cluster$ms_cluster_diagnosis <- gsub("AD,DLB", "other_dem", ms_cluster$ms_cluster_diagnosis)
ms_cluster$ms_cluster_diagnosis <- gsub("FTD,FTD-TDP,MND", "FTD", ms_cluster$ms_cluster_diagnosis)


table(ms_cluster[['ms_cluster_diagnosis']])
table(ms_cluster[['simplified_diagnosis']])
# ms_cluster[['ms_cluster_diagnosis']]

In [ ]:
DefaultAssay(ms_cluster) <- 'FLAT'
all.symptombins <- rownames(ms_cluster[["FLAT"]])
ms_cluster <- NormalizeData(ms_cluster,normalization.method = "LogNormalize", scale.factor = 100)
ms_cluster <- ScaleData(ms_cluster, features = all.symptombins,do.center = TRUE,scale.max = 10, model.use = "linear")
ms_cluster <- FindVariableFeatures(ms_cluster, selection.method = "vst", nfeatures = 1500)
ms_cluster <- perform_pca_and_visualization(ms_cluster,rname='pca')
print(ElbowPlot(ms_cluster,reduction='pca'))
ms_cluster <- JackStraw(ms_cluster, num.replicate = 100,assay='FLAT')
ms_cluster <- ScoreJackStraw(ms_cluster, dims = 1:20)
print(JackStrawPlot(ms_cluster, dims = 1:20))

In [ ]:
DefaultAssay(ms_cluster) <- 'TEMP'
all.symptombins <- rownames(ms_cluster[["TEMP"]])
ms_cluster <- NormalizeData(ms_cluster,normalization.method = "LogNormalize", scale.factor = 100)
ms_cluster <- ScaleData(ms_cluster, features = all.symptombins,do.center = TRUE,scale.max = 10, model.use = "linear")
ms_cluster <- FindVariableFeatures(ms_cluster, selection.method = "vst", nfeatures = 1500)
ms_cluster <- perform_pca_and_visualization(ms_cluster,rname='apca') ## change to pca for plots
# print(ElbowPlot(temp,reduction='apca')) ## change to pcs for plots
# temp <- JackStraw(temp, num.replicate = 100,assay='TEMP')
# temp <- ScoreJackStraw(temp, dims = 1:20)
# print(JackStrawPlot(temp, dims = 1:20))
print(ElbowPlot(ms_cluster,reduction='apca'))

In [ ]:
ms_cluster <- FindMultiModalNeighbors(
    ms_cluster,
    reduction.list = list("pca", "apca"),
    dims.list = list(1:3, 1:15), modality.weight.name = list("FLAT.weight","TEMP.weight")
    # dims.list = list(1:3, 1:7), modality.weight.name = list("FLAT.weight","TEMP.weight")
)

In [ ]:
ms_cluster <- RunUMAP(ms_cluster,
                      # dims = 1:10,
                      nn.name = "weighted.nn",
                      reduction.name = "wnn.umap",
                      reduction.key = "wnnUMAP_",
                      verbose=FALSE  
)
ms_cluster <- FindClusters(ms_cluster,
                              resolution = 0.3,
                              graph.name = "wsnn",
                              algorithm = 3,
                              verbose = TRUE)

In [ ]:
w = 12
h = 7.5
pticksize = 40
options(repr.plot.width = w, repr.plot.height = h)
p1 <- DimPlot(ms_cluster, reduction = 'wnn.umap', label = FALSE, repel = TRUE, shape.by = 'clin_coherence',
              label.size = 5,pt.size=4,cols="Dark2",shuffle=TRUE) +
    theme(axis.title.x = element_text(size = pticksize),
                axis.title.y = element_text(size = pticksize),
                legend.text = element_text(size = pticksize),
                axis.text.x = element_text(size = pticksize),
          # legend.position = "none",
               axis.text.y = element_text(size = pticksize))#+ #coord_cartesian(xlim = c(-3, 3.5), ylim = c(-3, 3))+
    # scale_x_continuous(breaks = c(-3,-2,-1, 0,1,2, 3)) + # Add x-axis tick marks at -3, 0, and 3+
    # scale_y_continuous(breaks = c(-3,-2,-1, 0,1,2, 3))  # Add x-axis tick marks at -3, 0, and 3


# p2 <- DimPlot(psych_cluster, reduction = 'wnn.umap', group.by = 'ms_cluster_diagnosis', label = TRUE, repel = TRUE, label.size = 2.5,cols=diagnosis_colors) 
p3 <- DimPlot(ms_cluster, reduction = 'wnn.umap', group.by = 'ms_cluster_diagnosis', label = FALSE, #cog_cluster_diagnosis
              raster = NULL,pt.size=4,shuffle=TRUE,seed=3,label.box=FALSE, shape.by = 'clin_coherence',
              repel = TRUE, label.size = 4,cols=diagnosis_colors) +
    theme(axis.title.x = element_text(size = pticksize),
               axis.title.y = element_text(size = pticksize),
               legend.text = element_text(size = pticksize),
          # legend.position = "none",
               axis.text.x = element_text(size = pticksize),
               axis.text.y = element_text(size = pticksize))+ #coord_cartesian(xlim = c(-3, 3.5), ylim = c(-3, 3))+
    # scale_x_continuous(breaks = c(-3,-2,-1, 0,1,2, 3)) + # Add x-axis tick marks at -3, 0, and 3+
    # scale_y_continuous(breaks = c(-3,-2,-1, 0,1,2, 3)) + # Add x-axis tick marks at -3, 0, and 3
    labs(title = "")  # Remove plot title
p1
# w = 12
# h = 10
options(repr.plot.width = w, repr.plot.height = h)
p3

# Save p1 to a PDF file
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/ms/dimplot_ms_cluster.pdf", plot = p1, width = w, height = h,dpi=300)

# Save p3 to a PDF file
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/ms/dimplot_ms_diagnosis.pdf", plot = p3, width = w, height = h,dpi=300)

In [ ]:
## add circle
# w = 10
# h = 10
unique_diagnoses <- table(ms_cluster$ms_cluster_diagnosis) #cog_cluster_diagnosis
unique_diagnoses
names(unique_diagnoses[unique_diagnoses > 10])
diagnoses <- c('MND','MND_other','MSA','CON','ATAXIA')
# diagnoses <- names(unique_diagnoses[unique_diagnoses > 10])
# diagnoses
options(repr.plot.width = w, repr.plot.height = h)
# ############### FUNCTION TO MAKE A DIMPLOT WITH CIRCLES AROUND DIAGNOSES OF INTEREST #######
# ## returns a plot and a list of donorids for each diagnosis that are 'outside' the circle
returns <- generate_dim_plot(ms_cluster, diagnoses,diag='simplified_diagnosis',p3) #cog_cluster_diagnosis
returns[[2]]
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/ms/dimplot_ms_diagnosis.pdf", plot = returns[[2]], width = w, height = h,dpi=300)

##### identity bars
for cog we do the coherence analysis

In [ ]:
w = 20
h = 7
options(repr.plot.width = w, repr.plot.height = h)
# p_values_df <- calculate_p_values(ms_cluster,diagnosis_column = "ms_cluster_diagnosis") #cog_cluster_diagnosis

# unique(rownames(p_values_df))
# corrected_p_values_df <- p_values_df %>%
#   mutate(across(everything(), ~ p.adjust(.x, method = "BH")))
# # p_values_df
# # corrected_p_values_df


# # diagnoses <- c('AD','FTD','MS','ATAXIA','DLB','MSA','PD','MND','PSP','CON','VD')
# # # diagnoses <- c('MS')
# diagnoses <- unique(rownames(p_values_df))
# # diagnoses <- names(unique_diagnoses[unique_diagnoses > 10])
# p_values_df_diag <- calculate_p_values_diag(ms_cluster,diagnosis_column = "ms_cluster_diagnosis",
#                                             variable_to_loop='clin_coherence',diagnoses,verbose=FALSE)

# # p_values_df_diag
# corrected_p_values_diag <- p_values_df_diag %>%
#   mutate(across(everything(), ~ p.adjust(.x, method = "BH")))
# # corrected_p_values_diag

# # # print(corrected_p_values_df)
# identitybars <- plot_cluster_diagnosis_diag(ms_cluster,corrected_p_values_df,
#                                             diagnosis_column = "ms_cluster_diagnosis",color='white',corrected_p_values_diag)     
# identitybars <- identitybars + theme(axis.title.x = element_text(size = 24),
#            axis.title.y = element_text(size = 24),
#            legend.text = element_text(size = 24),
#            axis.text.x = element_text(size = 24),
#            axis.text.y = element_text(size = 24))#+ coord_cartesian(xlim = c(-6, 6), ylim = c(-6, 6))
# identitybars
# ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/ms/identitybars_ms_cluster.pdf",
#        plot = identitybars, width = w, height = h,dpi=300)

# library(tidyverse)
# options(repr.plot.width = 12, repr.plot.height = 12)
# diagnoses <- c('AD','FTD','MS','ATAXIA','DLB','MSA','PD','MND','PSP','CON','VD','PSYCH')
table(ms_cluster@meta.data$ms_cluster_diagnosis)
diagnoses <- rownames(table(ms_cluster@meta.data$ms_cluster_diagnosis))
print(list(diagnoses))
known_clin_diag <-  c('ALS',"ATAXIA","CON","MS_undefined","MS,MS-PP","MS,MS-RR","MS,MS-SP","MSA","VD")
print(known_clin_diag)
# diagnoses <- c('MS')
p_values <- calculate_p_values_diag(ms_cluster,diagnosis_column = "ms_cluster_diagnosis",variable_to_loop='clin_coherence',diagnoses, known_clin_diag, verbose=FALSE)
# p_values_df_diag

p_values_diagnoses <- p_values$p_values
p_values_coherence <- p_values$p_values_diag

p_values_diagnoses <- p_values_diagnoses %>%
  spread(key = Cluster, value = P_Value)
rownames(p_values_diagnoses) <- p_values_diagnoses$Diagnosis
p_values_diagnoses <- p_values_diagnoses[, -1]
# print(p_values_diagnoses)
p_values_diagnoses <- p_values_diagnoses %>%
  mutate(across(everything(), ~ p.adjust(.x, method = "BH")))
p_values_diagnoses

p_values_coherence <- p_values_coherence %>%
  spread(key = Cluster, value = P_Value)
rownames(p_values_coherence) <- p_values_coherence$Diagnosis
p_values_coherence <- p_values_coherence[, -1]
# print(p_values_coherence)
p_values_coherence <- p_values_coherence %>%
  mutate(across(everything(), ~ p.adjust(.x, method = "BH")))
p_values_coherence



identitybars <- plot_cluster_diagnosis_diag(ms_cluster,p_values_diagnoses,diagnosis_column = "ms_cluster_diagnosis",color='white',p_values_coherence)     
identitybars <- identitybars + theme(axis.title.x = element_text(size = 24),
           axis.title.y = element_text(size = 24),
           legend.text = element_text(size = 24),
           axis.text.x = element_text(size = 24),
           axis.text.y = element_text(size = 24))#+ coord_cartesian(xlim = c(-6, 6), ylim = c(-6, 6))
identitybars
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/ms/identitybars_ms_cluster.pdf", plot = identitybars, width = w, height = h,dpi=300)




##### plot on original cluster

In [ ]:
w = 10
h = 7.5
options(repr.plot.width = w, repr.plot.height = h)
pticksize = 40
mscells <- WhichCells(merged_seurat, idents = c( "4"))
ms_highlight <- DimPlot(merged_seurat, reduction = 'wnn.umap',label = FALSE,cells.highlight = mscells,cols.highlight='black',sizes.highlight=2,
              raster = NULL,pt.size=1.5,shuffle=TRUE,seed=3,label.box=FALSE,
              repel = TRUE, label.size = 4)
ms_highlight
# Save to a PDF file
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/ms/highlight.pdf",
       plot = ms_highlight, width = w, height = h,dpi=300)


##### heatmap

In [ ]:
DefaultAssay(ms_cluster) <- 'FLAT'
extended_cluster_markers_flat = find_extended_cluster_markers(ms_cluster,0.05,method='all',assay='FLAT',verbose=FALSE) 

DefaultAssay(ms_cluster) <- 'TEMP'
extended_cluster_markers_temp = find_extended_cluster_markers(ms_cluster,0.05,method='all',assay='TEMP',verbose=FALSE) 

file_path <- "/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_analysis/clustermarkers/"
file_name_flat <- "msclustermarkerflattened.xlsx"
file_name_temp <- "msclustermarkertemporal.xlsx"
flat_path <- paste0(file_path, file_name_flat)
temp_path <- paste0(file_path, file_name_temp)
write.xlsx(extended_cluster_markers_flat, flat_path, row.names = FALSE)
write.xlsx(extended_cluster_markers_temp, temp_path, row.names = FALSE)


In [ ]:
returns <- create_marker_df(extended_cluster_markers_temp, extended_cluster_markers_flat, sup3)
w=8
h=10
combined_marker_list <- returns[[1]]
inge_markers <- annotation_df[annotation_df$Domain %in% c("Cognitive", "Psychiatric","General"), ]$ITname
inge_markers
# combined_marker_list
# # combined_marker_list
annotation_df <- returns[[2]]
# annotation_df
savepath_main = "/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/ms/heatmap_ms_cluster.pdf"
returns <- generate_heatmap_plot(ms_cluster,assay='FLAT', combined_marker_list, annotation_df,5,savepath=savepath_main)
mainmarkers <- "/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_analysis/clustermarkers/combined_msmarkers.txt"
writeLines(returns, con = mainmarkers)
returns

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 6)
VlnPlot(
  object = ms_cluster,
  features = 'Age',
  pt.size=0,  
  split.by='sex',
  split.plot=TRUE
  # group.by = 'seurat_clusters',
  # cols = viridis(length(unique(merged_cog$seurat_clusters)), option = 'D'),
)

# options(repr.plot.width = 7, repr.plot.height = 6)
# VlnPlot(
#   object = merged_cog,
#   features = 'File',
#   pt.size=0,  
#   split.by='sex',
#   split.plot=TRUE
#   # group.by = 'seurat_clusters',
#   # cols = viridis(length(unique(merged_cog$seurat_clusters)), option = 'D'),
# )


In [ ]:
# apoe <- data.frame(DonorID = rownames(ms_cluster@meta.data),
#                     seurat_clusters = ms_cluster@meta.data$seurat_clusters,
#                     diagnosis = ms_cluster@meta.data[['simplified_diagnosis']],
#                     quality =   ms_cluster@meta.data[['model_and_clinic3']],
#                     apoe = ms_cluster@meta.data[['APOE']]
# )    

# apoe <- apoe %>%
#   filter(seurat_clusters %in% c(0, 1)& !is.na(apoe)) %>%
#   group_by(seurat_clusters, apoe) %>%
#   summarise(count = n()) %>%
#   ungroup() %>%
#   group_by(seurat_clusters) %>%
#   mutate(total_count = sum(count),
#          percentage = (count / total_count) * 100)

# # print(apoe)

# apoe
# data <- matrix(c(7, 104 - 7,14.4, 5, 92 - 5,23.7), ncol = 2)
# rownames(data) <- c("APOE44", "Other Genotypes",'percentage (%)')
# colnames(data) <- c("Cluster 0", "Cluster 1")

# # Perform the chi-squared test
# chisq.test(data)
# data

##############################################################################################################
# ## to do: zoek de verschillen tussen MS-SP
# sp <- data.frame(cluster = Idents(ms_cluster), diagnosis = ms_cluster$ms_cluster_diagnosis,Age=ms_cluster@meta.data$Age)
# sp <- sp %>%
#   filter(diagnosis == "MS,MS-SP" )
# head(sp) 
# cluster_1 <- subset(sp, cluster == "1")
# cluster_2 <- subset(sp, cluster == "2")

# # Perform Mann-Whitney U test
# mwu_result <- wilcox.test(cluster_1$Age, cluster_2$Age)

# # Summarize Mann-Whitney U test results
# print(mwu_result)

# ggplot(sp, aes(x = cluster, y = Age, fill = cluster)) +
#   geom_violin() +
#   labs(x = "Cluster", y = "Age") + theme_minimal()

##############################################################################################################
# options(repr.plot.width = 7, repr.plot.height = 6)
# VlnPlot(
#   object = ms_cluster,
#   features = 'Age',
#   pt.size=0,  
#   split.by='sex',
#   split.plot=TRUE
#   # group.by = 'seurat_clusters',
#   # cols = viridis(length(unique(merged_cog$seurat_clusters)), option = 'D'),
# )
# ad <- data.frame(cluster = Idents(ms_cluster), Age=ms_cluster@meta.data$Age)
# mean_age_per_cluster <- ad %>%
#   group_by(cluster) %>%
#   summarise(mean_age = mean(Age),std_age = sd(Age),max=max(Age))
# pairwise_kruskal <- pairwise.wilcox.test(ad$Age, ad$cluster, p.adjust.method = "bonferroni")
# significant_pairs <- data.frame(from = character(), to = character(), p.value = numeric())
# print(mean_age_per_cluster)
# for (i in 1:nrow(pairwise_kruskal$p.value)) {
#   for (j in 1:ncol(pairwise_kruskal$p.value)) {
#     if (!is.na(pairwise_kruskal$p.value[i, j]) && pairwise_kruskal$p.value[i, j] < 0.05) {
#       significant_pairs <- rbind(significant_pairs, data.frame(from = rownames(pairwise_kruskal$p.value)[i],
#                                                                to = colnames(pairwise_kruskal$p.value)[j],
#                                                                p.value = pairwise_kruskal$p.value[i, j]))
#     }
#   }
# }
# significant_pairs$to = as.integer(significant_pairs$to)
# significant_pairs$dist = abs(as.integer(significant_pairs$to) - as.integer(significant_pairs$from))
# significant_pairs <- significant_pairs %>%
#   arrange(desc(dist),to)%>%
#   mutate(height = 150 - (row_number() -1) * 5)
# significant_pairs$p.star[significant_pairs$p.value < 0.05] <- "*"
# significant_pairs$p.star[significant_pairs$p.value < 0.005] <- "**"
# significant_pairs$p.star[significant_pairs$p.value < 0.0005] <- "***"
# significant_pairs#$to
# g <- VlnPlot(
#   object = ms_cluster,
#   features = 'Age',
#   pt.size=0,  
#   split.by='sex',
#   split.plot=TRUE
#   )+ scale_y_continuous(limits = c(0,150))+ geom_signif(
#     y_position = significant_pairs$height, xmin = significant_pairs$to+1, xmax = significant_pairs$from,
#     annotation = significant_pairs$p.star, tip_length = 0.02
#   )
# g

##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################

In [ ]:
# head(merged_seurat@meta.data)
ms_df <- data.frame(DonorID = rownames(ms_cluster@meta.data),ms_clusters = ms_cluster@meta.data$seurat_clusters)
# head(df2)
# seurat_clusters <- merge(df, cluster_df, by = "DonorID", all.x = TRUE)
ms <- merge(cogmain, ms_df, by = "DonorID", all.x = TRUE)
tail(ms)
write.csv(ms, file = '/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_analysis/data/seurat_output.csv', row.names = FALSE)


# PD cluster

In [ ]:
pd_cluster <- subset(merged_seurat, idents = c(1))
Assays(pd_cluster)

In [ ]:
# seurat_object[['simplified_diagnosis']] <- meta_data[match(names(seurat_object$orig.ident), rownames(meta_data)),'simplified_diagnosis']
pd_cluster$pd_cluster_diagnosis <- pd_cluster$diagnosis

# pd_cluster$pd_cluster_diagnosis <- gsub("AD,ENCEPHA,VE", "other_dem", pd_cluster$pd_cluster_diagnosis)
# pd_cluster$pd_cluster_diagnosis <- gsub("ATAXIA,SCA", "ATAXIA", pd_cluster$pd_cluster_diagnosis)
pd_cluster$pd_cluster_diagnosis <- gsub("DEM,ENCEPHA,VE", "other_dem", pd_cluster$pd_cluster_diagnosis)
# pd_cluster$pd_cluster_diagnosis <- gsub("AD,DLB", "other_dem", pd_cluster$pd_cluster_diagnosis)
pd_cluster$pd_cluster_diagnosis <- gsub("DEM,SICC", "other_dem", pd_cluster$pd_cluster_diagnosis)
# pd_cluster$pd_cluster_diagnosis <- gsub("PD,AD", "other_dem", pd_cluster$pd_cluster_diagnosis)
# pd_cluster$pd_cluster_diagnosis <- gsub("AD,CA", "other_dem", pd_cluster$pd_cluster_diagnosis)
# pd_cluster$pd_cluster_diagnosis <- gsub("CBD", "other_dem", pd_cluster$pd_cluster_diagnosis)
# pd_cluster$pd_cluster_diagnosis <- gsub("DLB,SICC", "FTD", pd_cluster$pd_cluster_diagnosis)
pd_cluster$pd_cluster_diagnosis <- gsub("FTD,FTD-TDP", "FTD-TDP", pd_cluster$pd_cluster_diagnosis)
pd_cluster$pd_cluster_diagnosis <- gsub("FTD,FTD-TDP,MND", "FTD", pd_cluster$pd_cluster_diagnosis)
pd_cluster$pd_cluster_diagnosis <- gsub("FTD,PID", "FTD", pd_cluster$pd_cluster_diagnosis)
pd_cluster$pd_cluster_diagnosis <- gsub("FTD,FTD-TAU,TAU", "FTD", pd_cluster$pd_cluster_diagnosis)
pd_cluster$pd_cluster_diagnosis <- gsub("FTD-TDP-A,PROG", "FTD", pd_cluster$pd_cluster_diagnosis)
pd_cluster$pd_cluster_diagnosis <- gsub("FTD-TDP-B,C9ORF72", "FTD", pd_cluster$pd_cluster_diagnosis)
pd_cluster$pd_cluster_diagnosis <- gsub("FTD-TDP,MND", "FTD", pd_cluster$pd_cluster_diagnosis)
pd_cluster$pd_cluster_diagnosis <- gsub("FTD-TDP_undefined", "FTD", pd_cluster$pd_cluster_diagnosis)
pd_cluster$pd_cluster_diagnosis <- gsub("ATAXIA,FA", "ATAXIA", pd_cluster$pd_cluster_diagnosis)
pd_cluster$pd_cluster_diagnosis <- gsub("ATAXIA,SCA", "ATAXIA", pd_cluster$pd_cluster_diagnosis)
pd_cluster$pd_cluster_diagnosis <- gsub("ATAXIA,FXTAS", "ATAXIA", pd_cluster$pd_cluster_diagnosis)

table(pd_cluster[['pd_cluster_diagnosis']])
table(pd_cluster[['simplified_diagnosis']])
# ms_cluster[['ms_cluster_diagnosis']]

# merged_cog$cog_cluster_diagnosis <- gsub("PSYCH,SCZ", "PSYCH", merged_cog$cog_cluster_diagnosis)
# merged_cog$cog_cluster_diagnosis <- gsub("PSYCH,BP", "PSYCH", merged_cog$cog_cluster_diagnosis)
# merged_cog$cog_cluster_diagnosis <- gsub("PSYCH,MDD", "PSYCH", merged_cog$cog_cluster_diagnosis)
# merged_cog$cog_cluster_diagnosis <- gsub("MS,MS-PP", "MS", merged_cog$cog_cluster_diagnosis)
# merged_cog$cog_cluster_diagnosis <- gsub("MS,MS-SP", "MS", merged_cog$cog_cluster_diagnosis)

# merged_cog$cog_cluster_diagnosis <- gsub("DEM,SICC,AGD", "DEM,SICC", merged_cog$cog_cluster_diagnosis)


# merged_cog$cog_cluster_diagnosis <- gsub("FTD,FTD-TDP_undefined", "FTD-TDP", merged_cog$cog_cluster_diagnosis)
# merged_cog$cog_cluster_diagnosis <- gsub("FTD,FTD-TDP-C", "FTD-TDP", merged_cog$cog_cluster_diagnosis)
# merged_cog$cog_cluster_diagnosis <- gsub("FTD,FTD-TDP-A,PROG", "FTD-TDP", merged_cog$cog_cluster_diagnosis)
# merged_cog$cog_cluster_diagnosis <- gsub("FTD,FTD-TDP-B,C9ORF72", "FTD-TDP", merged_cog$cog_cluster_diagnosis)



In [ ]:
DefaultAssay(pd_cluster) <- 'FLAT'
all.symptombins <- rownames(pd_cluster[["FLAT"]])
pd_cluster <- NormalizeData(pd_cluster,normalization.method = "LogNormalize", scale.factor = 100)
pd_cluster <- ScaleData(pd_cluster, features = all.symptombins,do.center = TRUE,scale.max = 10, model.use = "linear")
pd_cluster <- FindVariableFeatures(pd_cluster, selection.method = "vst", nfeatures = 1500)
pd_cluster <- perform_pca_and_visualization(pd_cluster,rname='pca')
print(ElbowPlot(pd_cluster,reduction='pca'))
pd_cluster <- JackStraw(pd_cluster, num.replicate = 100,assay='FLAT')
pd_cluster <- ScoreJackStraw(pd_cluster, dims = 1:20)
print(JackStrawPlot(pd_cluster, dims = 1:20))

In [ ]:
DefaultAssay(pd_cluster) <- 'TEMP'
all.symptombins <- rownames(pd_cluster[["TEMP"]])
pd_cluster <- NormalizeData(pd_cluster,normalization.method = "LogNormalize", scale.factor = 100)
pd_cluster <- ScaleData(pd_cluster, features = all.symptombins,do.center = TRUE,scale.max = 10, model.use = "linear")
pd_cluster <- FindVariableFeatures(pd_cluster, selection.method = "vst", nfeatures = 1500)
pd_cluster <- perform_pca_and_visualization(pd_cluster,rname='apca') ## change to pca for plots
print(ElbowPlot(pd_cluster,reduction='apca')) ## change to pcs for plots
# pd_cluster <- JackStraw(pd_cluster, num.replicate = 100,assay='TEMP')
# pd_cluster <- ScoreJackStraw(pd_cluster, dims = 1:20)
# print(JackStrawPlot(pd_cluster, dims = 1:20))

In [ ]:
pd_cluster <- FindMultiModalNeighbors(
    pd_cluster,
    reduction.list = list("pca", "apca"),
    # dims.list = list(1:5, 1:25), modality.weight.name = list("FLAT.weight","TEMP.weight")
    dims.list = list(1:7, 1:8), modality.weight.name = list("FLAT.weight","TEMP.weight")
)

In [ ]:
pd_cluster <- RunUMAP(pd_cluster,
                      # dims = 1:10,
                      nn.name = "weighted.nn",
                      reduction.name = "wnn.umap",
                      reduction.key = "wnnUMAP_",
                      verbose=FALSE  
)


In [ ]:
pd_cluster <- FindClusters(pd_cluster,
                              resolution = 0.4,
                              graph.name = "wsnn",
                              algorithm = 3,
                              verbose = TRUE)

In [ ]:
w = 12
h = 7.5
pticksize = 40
options(repr.plot.width = w, repr.plot.height = h)
p1 <- DimPlot(pd_cluster, reduction = 'wnn.umap', label = FALSE, repel = TRUE,shape.by = 'clin_coherence',
              label.size = 5,pt.size=4,cols="Dark2",shuffle=TRUE) +
    theme(axis.title.x = element_text(size = pticksize),
                axis.title.y = element_text(size = pticksize),
                legend.text = element_text(size = pticksize),
                axis.text.x = element_text(size = pticksize),
          # legend.position = "none",
               axis.text.y = element_text(size = pticksize))+ coord_cartesian(xlim = c(-4.5, 4))+
    scale_x_continuous(breaks = c(-5,-4,-3,-2,-1, 0,1,2,3, 4)) # Add x-axis tick marks at -3, 0, and 3+
    # scale_y_continuous(breaks = c(-3,-2,-1, 0,1,2, 3))  # Add x-axis tick marks at -3, 0, and 3


# p2 <- DimPlot(psych_cluster, reduction = 'wnn.umap', group.by = 'ms_cluster_diagnosis', label = TRUE, repel = TRUE, label.size = 2.5,cols=diagnosis_colors) 
p3 <- DimPlot(pd_cluster, reduction = 'wnn.umap', group.by = 'pd_cluster_diagnosis', label = FALSE, #cog_cluster_diagnosis
              raster = NULL,pt.size=4,shuffle=TRUE,seed=3,label.box=FALSE, shape.by = 'clin_coherence',
              repel = TRUE, label.size = 4,cols=diagnosis_colors) +
    theme(axis.title.x = element_text(size = pticksize),
               axis.title.y = element_text(size = pticksize),
               legend.text = element_text(size = pticksize),
          legend.position = "none",
               axis.text.x = element_text(size = pticksize),
               axis.text.y = element_text(size = pticksize))+ coord_cartesian(xlim = c(-4.5, 4))+
    scale_x_continuous(breaks = c(-5,-4,-3,-2,-1, 0,1,2,3, 4))+  # Add x-axis tick marks at -3, 0, and 3+
    # scale_y_continuous(breaks = c(-3,-2,-1, 0,1,2, 3)) + # Add x-axis tick marks at -3, 0, and 3
    labs(title = "")  # Remove plot title
p1
# w = 12
# h = 10
options(repr.plot.width = w, repr.plot.height = h)
p3

# Save p1 to a PDF file
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/pd/dimplot_pd_cluster.pdf",
       plot = p1, width = w, height = h,dpi=300)

# Save p3 to a PDF file
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/pd/dimplot_pd_diagnosis.pdf",
       plot = p3, width = w, height = h,dpi=300)

In [ ]:
## add circle
# w = 10
# h = 7.5
unique_diagnoses <- table(pd_cluster$simplified_diagnosis) #cog_cluster_diagnosis
unique_diagnoses <- table(pd_cluster$pd_cluster_diagnosis)
unique_diagnoses
unique_diagnoses <- names(unique_diagnoses[unique_diagnoses > 10])
diagnoses <- c('AD,DLB','DLB,SICC','PSP','MSA','AD,CA','PD,AD')
# diagnoses <- c('DLB',)
# diagnoses <- names(unique_diagnoses[unique_diagnoses > 10])
options(repr.plot.width = w, repr.plot.height = h)
# # ############### FUNCTION TO MAKE A DIMPLOT WITH CIRCLES AROUND DIAGNOSES OF INTEREST #######
# # ## returns a plot and a list of donorids for each diagnosis that are 'outside' the circle
returns <- generate_dim_plot(pd_cluster, diagnoses,diag='pd_cluster_diagnosis',p3) #cog_cluster_diagnosis
returns[[2]] + scale_x_continuous(breaks = c(-5,-4,-3,-2,-1, 0,1,2,3, 4)) # Add x-axis tick marks at -3, 0, and 3+
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/pd/dimplot_pd_diagnosis.pdf",
       plot = returns[[2]], width = w, height = h,dpi=300)

##### plot on original cluster

In [ ]:
w = 10
h = 7.5
options(repr.plot.width = w, repr.plot.height = h)
pticksize = 40
pdcells <- WhichCells(merged_seurat, idents = c( "1"))
pd_highlight <- DimPlot(merged_seurat, reduction = 'wnn.umap',label = FALSE,cells.highlight = pdcells,
                        cols.highlight='black',sizes.highlight=2,raster = NULL,pt.size=1.5,shuffle=TRUE,
                        seed=3,label.box=FALSE,repel = TRUE, label.size = 4)
pd_highlight
# Save to a PDF file
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/pd/highlight.pdf",
       plot = pd_highlight, width = w, height = h,dpi=300)


##### identity bars
for cog we do the coherence analysis

In [ ]:
w = 7
h = 7
options(repr.plot.width = w, repr.plot.height = h)
# p_values_df <- calculate_p_values(pd_cluster,diagnosis_column = "simplified_diagnosis") #cog_cluster_diagnosis
# p_values_df <- calculate_p_values(pd_cluster,diagnosis_column = "pd_cluster_diagnosis") #cog_cluster_diagnosis
# unique(rownames(p_values_df))
# corrected_p_values_df <- p_values_df %>%
#   mutate(across(everything(), ~ p.adjust(.x, method = "BH")))
# p_values_df
# corrected_p_values_df


# diagnoses <- c('AD','FTD','ATAXIA','DLB','MSA','PD','PSP','CON','VD')
# # # diagnoses <- c('MS')
# diagnoses <- unique(rownames(p_values_df))
# diagnoses
# # diagnoses <- names(unique_diagnoses[unique_diagnoses > 10])
# p_values_df_diag <- calculate_p_values_diag(pd_cluster,diagnosis_column = "pd_cluster_diagnosis",
#                                             variable_to_loop='model_and_clinic3',diagnoses,verbose=FALSE)

# p_values_df_diag
# corrected_p_values_diag <- p_values_df_diag %>%
#   mutate(across(everything(), ~ p.adjust(.x, method = "BH")))
# # corrected_p_values_diag

# # # print(corrected_p_values_df)
# identitybars <- plot_cluster_diagnosis_diag(pd_cluster,corrected_p_values_df,
#                                             diagnosis_column = "pd_cluster_diagnosis",color='white',corrected_p_values_diag)     
# identitybars <- identitybars + theme(axis.title.x = element_text(size = 24),
#            axis.title.y = element_text(size = 24),
#            legend.text = element_text(size = 24),
#            axis.text.x = element_text(size = 24),
#            axis.text.y = element_text(size = 24))#+ coord_cartesian(xlim = c(-6, 6), ylim = c(-6, 6))
# identitybars
# ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/pd/identitybars_pd_cluster.pdf",
#        plot = identitybars, width = w, height = h,dpi=300)


table(pd_cluster@meta.data$pd_cluster_diagnosis)
diagnoses <- rownames(table(pd_cluster@meta.data$pd_cluster_diagnosis))
print(list(diagnoses))
known_clin_diag <-  c('AD',"AD,DLB","DLB","FTD","MSA","PD","PSP")
print(known_clin_diag)
# diagnoses <- c('MS')
p_values <- calculate_p_values_diag(pd_cluster,diagnosis_column = "pd_cluster_diagnosis",variable_to_loop='clin_coherence',diagnoses, known_clin_diag, verbose=FALSE)
# p_values_df_diag

p_values_diagnoses <- p_values$p_values
p_values_coherence <- p_values$p_values_diag

p_values_diagnoses <- p_values_diagnoses %>%
  spread(key = Cluster, value = P_Value)
rownames(p_values_diagnoses) <- p_values_diagnoses$Diagnosis
p_values_diagnoses <- p_values_diagnoses[, -1]
# print(p_values_diagnoses)
p_values_diagnoses <- p_values_diagnoses %>%
  mutate(across(everything(), ~ p.adjust(.x, method = "BH")))
p_values_diagnoses

p_values_coherence <- p_values_coherence %>%
  spread(key = Cluster, value = P_Value)
rownames(p_values_coherence) <- p_values_coherence$Diagnosis
p_values_coherence <- p_values_coherence[, -1]
# print(p_values_coherence)
p_values_coherence <- p_values_coherence %>%
  mutate(across(everything(), ~ p.adjust(.x, method = "BH")))
p_values_coherence



identitybars <- plot_cluster_diagnosis_diag(pd_cluster,p_values_diagnoses,diagnosis_column = "pd_cluster_diagnosis",color='white',p_values_coherence)     
identitybars <- identitybars + theme(axis.title.x = element_text(size = 24),
           axis.title.y = element_text(size = 24),
           legend.text = element_text(size = 24),
           axis.text.x = element_text(size = 24),
           axis.text.y = element_text(size = 24))#+ coord_cartesian(xlim = c(-6, 6), ylim = c(-6, 6))
identitybars
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/pd/identitybars_pd_cluster.pdf", plot = identitybars, width = w, height = h,dpi=300)



##### heatmap

In [ ]:
DefaultAssay(pd_cluster) <- 'FLAT'
extended_cluster_markers_flat = find_extended_cluster_markers(pd_cluster,0.05,method='all',assay='FLAT',verbose=FALSE) 

DefaultAssay(pd_cluster) <- 'TEMP'
extended_cluster_markers_temp = find_extended_cluster_markers(pd_cluster,0.05,method='all',assay='TEMP',verbose=FALSE) 

file_path <- "/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_analysis/clustermarkers/"
file_name_flat <- "pdclustermarkerflattened.xlsx"
file_name_temp <- "pdclustermarkertemporal.xlsx"
flat_path <- paste0(file_path, file_name_flat)
temp_path <- paste0(file_path, file_name_temp)
write.xlsx(extended_cluster_markers_flat, flat_path, row.names = FALSE)
write.xlsx(extended_cluster_markers_temp, temp_path, row.names = FALSE)


In [ ]:
returns <- create_marker_df(extended_cluster_markers_temp, extended_cluster_markers_flat, sup3)
w=8
h=10
combined_marker_list <- returns[[1]]
inge_markers <- annotation_df[annotation_df$Domain %in% c("Cognitive", "Psychiatric","General"), ]$ITname
# inge_markers
# combined_marker_list
# # combined_marker_list
annotation_df <- returns[[2]]
# annotation_df
savepath_main = "/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/pd/heatmap_pd_cluster.pdf"
returns <- generate_heatmap_plot(pd_cluster,assay='FLAT', combined_marker_list, annotation_df,10,savepath=savepath_main)
mainmarkers <- "/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_analysis/clustermarkers/combined_pdmarkers.txt"
writeLines(returns, con = mainmarkers)
returns

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 6)
VlnPlot(
  object = pd_cluster,
  features = 'Age',
  pt.size=0,  
  split.by='sex',
  split.plot=TRUE
  # group.by = 'seurat_clusters',
  # cols = viridis(length(unique(merged_cog$seurat_clusters)), option = 'D'),
)

# options(repr.plot.width = 7, repr.plot.height = 6)
# VlnPlot(
#   object = merged_cog,
#   features = 'File',
#   pt.size=0,  
#   split.by='sex',
#   split.plot=TRUE
#   # group.by = 'seurat_clusters',
#   # cols = viridis(length(unique(merged_cog$seurat_clusters)), option = 'D'),
# )


In [ ]:
# head(merged_seurat@meta.data)
pd_df <- data.frame(DonorID = rownames(pd_cluster@meta.data),pd_clusters = pd_cluster@meta.data$seurat_clusters)
# head(df2)
# seurat_clusters <- merge(df, cluster_df, by = "DonorID", all.x = TRUE)
pd <- merge(ms,pd_df, by = "DonorID", all.x = TRUE)
tail(pd)
write.csv(pd, file = '/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_analysis/data/seurat_output.csv', row.names = FALSE)


In [ ]:
# options(repr.plot.width = 7, repr.plot.height = 6)
# VlnPlot(
#   object = pd_cluster,
#   features = 'Age',
#   pt.size=0,  
#   split.by='sex',
#   split.plot=TRUE
#   # group.by = 'seurat_clusters',
#   # cols = viridis(length(unique(merged_cog$seurat_clusters)), option = 'D'),
# )

# # options(repr.plot.width = 7, repr.plot.height = 6)
# # VlnPlot(
# #   object = pd_cluster,
# #   features = 'File',
# #   pt.size=0,  
# #   split.by='sex',
# #   split.plot=TRUE
# #   # group.by = 'seurat_clusters',
# #   # cols = viridis(length(unique(merged_cog$seurat_clusters)), option = 'D'),
# # )
# ad <- data.frame(cluster = Idents(pd_cluster), Age=pd_cluster@meta.data$Age)
# mean_age_per_cluster <- ad %>%
#   group_by(cluster) %>%
#   summarise(mean_age = mean(Age),std_age = sd(Age),max=max(Age))
# pairwise_kruskal <- pairwise.wilcox.test(ad$Age, ad$cluster, p.adjust.method = "bonferroni")
# significant_pairs <- data.frame(from = character(), to = character(), p.value = numeric())
# print(mean_age_per_cluster)
# for (i in 1:nrow(pairwise_kruskal$p.value)) {
#   for (j in 1:ncol(pairwise_kruskal$p.value)) {
#     if (!is.na(pairwise_kruskal$p.value[i, j]) && pairwise_kruskal$p.value[i, j] < 0.05) {
#       significant_pairs <- rbind(significant_pairs, data.frame(from = rownames(pairwise_kruskal$p.value)[i],
#                                                                to = colnames(pairwise_kruskal$p.value)[j],
#                                                                p.value = pairwise_kruskal$p.value[i, j]))
#     }
#   }
# }
# significant_pairs$to = as.integer(significant_pairs$to)
# significant_pairs$dist = abs(as.integer(significant_pairs$to) - as.integer(significant_pairs$from))
# significant_pairs <- significant_pairs %>%
#   arrange(desc(dist),to)%>%
#   mutate(height = 150 - (row_number() -1) * 5)
# significant_pairs$p.star[significant_pairs$p.value < 0.05] <- "*"
# significant_pairs$p.star[significant_pairs$p.value < 0.005] <- "**"
# significant_pairs$p.star[significant_pairs$p.value < 0.0005] <- "***"
# significant_pairs#$to
# g <- VlnPlot(
#   object = pd_cluster,
#   features = 'Age',
#   pt.size=0,  
#   split.by='sex',
#   split.plot=TRUE
#   )+ scale_y_continuous(limits = c(0,150))+ geom_signif(
#     y_position = significant_pairs$height, xmin = significant_pairs$to+1, xmax = significant_pairs$from,
#     annotation = significant_pairs$p.star, tip_length = 0.02
#   )
# g

#############################################################################################################
# ## to do: zoek de verschillen tussen MS-SP
# pd <- data.frame(cluster = Idents(pd_cluster), diagnosis = pd_cluster$simplified_diagnosis,Age=pd_cluster@meta.data$Age)
# pd <- pd %>%
#   filter(diagnosis == "PD" )
# head(pd) 
# mean_age_per_cluster <- pd %>%
#   group_by(cluster) %>%
#   summarise(mean_age = mean(Age),std_age = sd(Age))

# print(mean_age_per_cluster)
# cluster_1 <- subset(pd, cluster == "0")
# cluster_2 <- subset(pd, cluster == "1")

# # Perform Mann-Whitney U test
# mwu_result <- wilcox.test(cluster_1$Age, cluster_2$Age)

# # Summarize Mann-Whitney U test results
# print(mwu_result)

# ggplot(pd, aes(x = cluster, y = Age, fill = cluster)) +
#   geom_violin() +
#   labs(x = "Cluster", y = "Age") + theme_minimal()


### PSYCH cluster

In [ ]:
psych_cluster <- subset(merged_seurat, idents = c(5))
Assays(psych_cluster)

In [ ]:
# seurat_object[['simplified_diagnosis']] <- meta_data[match(names(seurat_object$orig.ident), rownames(meta_data)),'simplified_diagnosis']
psych_cluster$psych_cluster_diagnosis <- psych_cluster$diagnosis


psych_cluster$psych_cluster_diagnosis <- gsub("AD,DLB", "other_dem", psych_cluster$psych_cluster_diagnosis)
psych_cluster$psych_cluster_diagnosis <- gsub("DLB,SICC", "FTD", psych_cluster$psych_cluster_diagnosis)
psych_cluster$psych_cluster_diagnosis <- gsub("FTD,FTD-TDP", "FTD", psych_cluster$psych_cluster_diagnosis)
psych_cluster$psych_cluster_diagnosis <- gsub("FTD,FTD-TAU,TAU", "FTD", psych_cluster$psych_cluster_diagnosis)
psych_cluster$psych_cluster_diagnosis <- gsub("FTD-A,PROG", "FTD", psych_cluster$psych_cluster_diagnosis)
psych_cluster$psych_cluster_diagnosis <- gsub("MS,MS-PP", "MS", psych_cluster$psych_cluster_diagnosis)
psych_cluster$psych_cluster_diagnosis <- gsub("MS,MS-SP", "MS", psych_cluster$psych_cluster_diagnosis)
psych_cluster$psych_cluster_diagnosis <- gsub("MND,ALS", "MND", psych_cluster$psych_cluster_diagnosis)
psych_cluster$psych_cluster_diagnosis <- gsub("FTD,FTD-TDP_undefined", "FTD", psych_cluster$psych_cluster_diagnosis)
psych_cluster$psych_cluster_diagnosis <- gsub("FTD_undefined", "FTD", psych_cluster$psych_cluster_diagnosis)
psych_cluster$psych_cluster_diagnosis <- gsub("MS_undefined", "MS", psych_cluster$psych_cluster_diagnosis)

table(psych_cluster[['psych_cluster_diagnosis']])
table(psych_cluster[['simplified_diagnosis']])
# ms_cluster[['ms_cluster_diagnosis']]

In [ ]:
DefaultAssay(psych_cluster) <- 'FLAT'
all.symptombins <- rownames(psych_cluster[["FLAT"]])
psych_cluster <- NormalizeData(psych_cluster,normalization.method = "LogNormalize", scale.factor = 100)
psych_cluster <- ScaleData(psych_cluster, features = all.symptombins,do.center = TRUE,scale.max = 10, model.use = "linear")
psych_cluster <- FindVariableFeatures(psych_cluster, selection.method = "vst", nfeatures = 1500)
psych_cluster <- perform_pca_and_visualization(psych_cluster,rname='pca')
print(ElbowPlot(psych_cluster,reduction='pca'))
psych_cluster <- JackStraw(psych_cluster, num.replicate = 100,assay='FLAT')
psych_cluster <- ScoreJackStraw(psych_cluster, dims = 1:20)
print(JackStrawPlot(psych_cluster, dims = 1:20))

In [ ]:
DefaultAssay(psych_cluster) <- 'TEMP'
all.symptombins <- rownames(psych_cluster[["TEMP"]])
psych_cluster <- NormalizeData(psych_cluster,normalization.method = "LogNormalize", scale.factor = 100)
psych_cluster <- ScaleData(psych_cluster, features = all.symptombins,do.center = TRUE,scale.max = 10, model.use = "linear")
psych_cluster <- FindVariableFeatures(psych_cluster, selection.method = "vst", nfeatures = 1500)
psych_cluster <- perform_pca_and_visualization(psych_cluster,rname='apca') ## change to pca for plots
print(ElbowPlot(psych_cluster,reduction='apca')) ## change to pcs for plots
# psych_cluster <- JackStraw(psych_cluster, num.replicate = 100,assay='TEMP')
# psych_cluster <- ScoreJackStraw(psych_cluster, dims = 1:20)
# print(JackStrawPlot(psych_cluster, dims = 1:20))

In [ ]:
psych_cluster <- FindMultiModalNeighbors(
    psych_cluster,
    reduction.list = list("pca", "apca"), #, "apca"
    # dims.list = list(1:5, c(2:7)),#
    # dims.list = list(1:4, 1:4),#
    dims.list = list(1:4, 1:4),#
    k.nn = 8,
    knn.range = 120,
    modality.weight.name = list("FLAT.weight","TEMP.weight") #
)

# DefaultAssay(psych_cluster) <- 'FLAT'
# DimPlot(psych_cluster, reduction = "pca")
# psych_cluster <- FindNeighbors(psych_cluster, dims = 1:5,reduction='pca')
# psych_cluster <- FindClusters(psych_cluster, reduction='pca', resolution = 0.5)
# psych_cluster <- RunUMAP(psych_cluster, dims = 1:5)
# DimPlot(psych_cluster, reduction = "umap")

In [ ]:
psych_cluster <- RunUMAP(psych_cluster,
                      # dims = 1:10,
                      nn.name = "weighted.nn",
                      reduction.name = "wnn.umap",
                      reduction.key = "wnnUMAP_",
                      verbose=FALSE  
)


In [ ]:
psych_cluster <- FindClusters(psych_cluster,
                              resolution = 0.3,
                              graph.name = "wsnn",
                              algorithm = 3,
                              verbose = TRUE)

In [ ]:
colnames(psych_cluster@meta.data)
# Assuming your DataFrame is named df
selected_rows <- psych_cluster@meta.data[psych_cluster@meta.data$psych_cluster_diagnosis == 'FTD_undefined', ]
selected_rows[,'diagnosis' ]

In [ ]:
w = 12
h = 7.5
pticksize = 40
options(repr.plot.width = w, repr.plot.height = h)
p1 <- DimPlot(psych_cluster, reduction = 'wnn.umap', label = FALSE,shape.by = 'clin_coherence',
              repel = TRUE, label.size = 5,pt.size=4,cols="Dark2",shuffle=TRUE) +
            theme(axis.title.x = element_text(size = pticksize),
                axis.title.y = element_text(size = pticksize),
                legend.text = element_text(size = pticksize),
                axis.text.x = element_text(size = pticksize),
          # legend.position = "none",
               axis.text.y = element_text(size = pticksize))#+ #coord_cartesian(xlim = c(-3, 3.5), ylim = c(-3, 3))+
    # scale_x_continuous(breaks = c(-3,-2,-1, 0,1,2, 3)) + # Add x-axis tick marks at -3, 0, and 3+
    # scale_y_continuous(breaks = c(-3,-2,-1, 0,1,2, 3))  # Add x-axis tick marks at -3, 0, and 3


# p2 <- DimPlot(psych_cluster, reduction = 'wnn.umap', group.by = 'ms_cluster_diagnosis', label = TRUE, repel = TRUE, label.size = 2.5,cols=diagnosis_colors) 
p3 <- DimPlot(psych_cluster, reduction = 'wnn.umap', group.by = 'psych_cluster_diagnosis', label = FALSE, #cog_cluster_diagnosis
              raster = NULL,pt.size=3,shuffle=TRUE,seed=3,label.box=FALSE,# shape.by = 'model_and_clinic3',
              repel = TRUE, label.size = 4,cols=diagnosis_colors) +
    theme(axis.title.x = element_text(size = pticksize),
               axis.title.y = element_text(size = pticksize),
               legend.text = element_text(size = pticksize),
          legend.position = "none",
               axis.text.x = element_text(size = pticksize),
               axis.text.y = element_text(size = pticksize))+ #coord_cartesian(xlim = c(-3, 3.5), ylim = c(-3, 3))+
    # scale_x_continuous(breaks = c(-3,-2,-1, 0,1,2, 3)) + # Add x-axis tick marks at -3, 0, and 3+
    # scale_y_continuous(breaks = c(-3,-2,-1, 0,1,2, 3)) + # Add x-axis tick marks at -3, 0, and 3
    labs(title = "")  # Remove plot title
p1
# w = 20
# h = 12
options(repr.plot.width = w, repr.plot.height = h)
p3

# Save p1 to a PDF file
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/psych/dimplot_psych_cluster.pdf", plot = p1, width = w, height = h,dpi=300)

# Save p3 to a PDF file
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/psych/dimplot_psych_diagnosis.pdf", plot = p3, width = w, height = h,dpi=300)

In [ ]:

## add circle
# w = 20
# h = 12
diagnoses <- table(psych_cluster$psych_cluster_diagnosis) #cog_cluster_diagnosis
diagnoses
diagnoses <- c('PSYCH,PTSD','PSYCH,SCZ','CON','PSYCH,BP','PSYCH,OCD','PSYCH,ASD','PSYCH,MDD')
# diagnoses <- names(unique_diagnoses[unique_diagnoses > 30])

diagnoses
options(repr.plot.width = w, repr.plot.height = h)
# ############### FUNCTION TO MAKE A DIMPLOT WITH CIRCLES AROUND DIAGNOSES OF INTEREST #######
# ## returns a plot and a list of donorids for each diagnosis that are 'outside' the circle
returns <- generate_dim_plot(psych_cluster, diagnoses,diag='psych_cluster_diagnosis',p3) #cog_cluster_diagnosis
returns[[2]]
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/psych/dimplot_psych_diagnosis.pdf", plot = returns[[2]], width = w, height = h,dpi=300)





##### plot on original cluster

In [ ]:
# copy_of_merged <- merged_seurat
# # copy_of_merged
# # colnames(copy_of_merged@meta.data)
# # colnames(psych_cluster@meta.data)
# original_rownames <- colnames(copy_of_merged@meta.data)
# copy_of_merged@meta.data <- merge(copy_of_merged@meta.data, psych_cluster@meta.data[, c("seurat_clusters","psych_cluster_diagnosis","simplified_diagnosis"), drop = FALSE], by = "row.names", all.x = TRUE)
# colnames(copy_of_merged@meta.data)[colnames(copy_of_merged@meta.data) == "seurat_clusters.y"] <- "psych_clusters"
# colnames(copy_of_merged@meta.data)[colnames(copy_of_merged@meta.data) == "seurat_clusters.x"] <- "seurat_clusters"
# colnames(copy_of_merged@meta.data)[colnames(copy_of_merged@meta.data) == "simplified_diagnosis.y"] <- "cog_simplified"
# colnames(copy_of_merged@meta.data)[colnames(copy_of_merged@meta.data) == "simplified_diagnosis.x"] <- "simplified_diagnosis"
# rownames(copy_of_merged@meta.data) <- copy_of_merged@meta.data$Row.names
# copy_of_merged@meta.data$Row.names <- NULL
# # original_rownames
# # head(copy_of_merged@meta.data)
# head(copy_of_merged@meta.data[copy_of_merged@meta.data$seurat_clusters == 5, ])

##### identity bars
for psych we dont have clin diag to compare with, so we do not do the coherence analysis

In [ ]:
w = 15
h = 7
# options(repr.plot.width = w, repr.plot.height = h)
# p_values_df <- calculate_p_values(psych_cluster,diagnosis_column = "psych_cluster_diagnosis")
# p_values_df
# corrected_p_values_df <- p_values_df %>%
#   mutate(across(everything(), ~ p.adjust(.x, method = "BH")))
# corrected_p_values_df
# # unique(rownames(p_values_df))
# diagnoses <- c('PSYCH,ASD','PSYCH,BP','PSYCH,MDD','PSYCH,OCD','PSYCH,PTSD','PSYCH,SCZ','CON')
# diagnoses <- unique(rownames(p_values_df))

# # p_values_df <- p_values_df[rownames(p_values_df) %in% diagnoses, ]


# # corrected_p_values_df
# identity_psych <- plot_cluster_diagnosis(psych_cluster,p_values_df,diagnosis_column = "psych_cluster_diagnosis",color='white')
# identity_psych
# ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/psych/identitybars_psych_cluster.pdf",
#        plot = identity_psych, width = w, height = h,dpi=300)


table(psych_cluster@meta.data$psych_cluster_diagnosis)
diagnoses <- rownames(table(psych_cluster@meta.data$psych_cluster_diagnosis))
print(list(diagnoses))
known_clin_diag <-  c('AD',"AD,DLB","DLB","FTD","MSA","PD","PSP")
print(known_clin_diag)
# diagnoses <- c('MS')
p_values <- calculate_p_values_diag(psych_cluster,diagnosis_column = "psych_cluster_diagnosis",variable_to_loop='clin_coherence',diagnoses, known_clin_diag, verbose=FALSE)
# p_values_df_diag

p_values_diagnoses <- p_values$p_values
p_values_coherence <- p_values$p_values_diag

p_values_diagnoses <- p_values_diagnoses %>%
  spread(key = Cluster, value = P_Value)
rownames(p_values_diagnoses) <- p_values_diagnoses$Diagnosis
p_values_diagnoses <- p_values_diagnoses[, -1]
# print(p_values_diagnoses)
p_values_diagnoses <- p_values_diagnoses %>%
  mutate(across(everything(), ~ p.adjust(.x, method = "BH")))
p_values_diagnoses

p_values_coherence <- p_values_coherence %>%
  spread(key = Cluster, value = P_Value)
rownames(p_values_coherence) <- p_values_coherence$Diagnosis
p_values_coherence <- p_values_coherence[, -1]
# print(p_values_coherence)
p_values_coherence <- p_values_coherence %>%
  mutate(across(everything(), ~ p.adjust(.x, method = "BH")))
p_values_coherence



identitybars <- plot_cluster_diagnosis_diag(psych_cluster,p_values_diagnoses,diagnosis_column = "psych_cluster_diagnosis",color='white',p_values_coherence)     
identitybars <- identitybars + theme(axis.title.x = element_text(size = 24),
           axis.title.y = element_text(size = 24),
           legend.text = element_text(size = 24),
           axis.text.x = element_text(size = 24),
           axis.text.y = element_text(size = 24))#+ coord_cartesian(xlim = c(-6, 6), ylim = c(-6, 6))
identitybars
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/psych/identitybars_psych_cluster.pdf", plot = identitybars, width = w, height = h,dpi=300)



##### heatmap

In [ ]:
DefaultAssay(psych_cluster) <- 'FLAT'
extended_cluster_markers_flat = find_extended_cluster_markers(psych_cluster,0.05,method='all',assay='FLAT',verbose=FALSE) 

DefaultAssay(psych_cluster) <- 'TEMP'
extended_cluster_markers_temp = find_extended_cluster_markers(psych_cluster,0.05,method='all',assay='TEMP',verbose=FALSE) 

file_path <- "/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_analysis/clustermarkers/"
file_name_flat <- "psychclustermarkerflattened.xlsx"
file_name_temp <- "psychclustermarkertemporal.xlsx"
flat_path <- paste0(file_path, file_name_flat)
temp_path <- paste0(file_path, file_name_temp)
write.xlsx(extended_cluster_markers_flat, flat_path, row.names = FALSE)
write.xlsx(extended_cluster_markers_temp, temp_path, row.names = FALSE)

In [ ]:
returns <- create_marker_df(extended_cluster_markers_temp, extended_cluster_markers_flat, sup3)
w=8
h=10
combined_marker_list <- returns[[1]]
# combined_marker_list
# # combined_marker_list
annotation_df <- returns[[2]]
# annotation_df
savepath_main = "/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/psych/heatmap_psych_cluster.pdf"
returns <- generate_heatmap_plot(psych_cluster,assay='FLAT', combined_marker_list, annotation_df,6,savepath=savepath_main)
mainmarkers <- "/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_analysis/clustermarkers/combined_psychmarkers.txt"
writeLines(returns, con = mainmarkers)
returns

In [ ]:
# head(merged_seurat@meta.data)
psych_df <- data.frame(DonorID = rownames(psych_cluster@meta.data),psych_clusters = psych_cluster@meta.data$seurat_clusters)
# head(df2)
# seurat_clusters <- merge(df, cluster_df, by = "DonorID", all.x = TRUE)
psych <- merge(pd, psych_df, by = "DonorID", all.x = TRUE)
tail(psych)
write.csv(psych, file = '/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_analysis/data/seurat_output.csv', row.names = FALSE)


In [ ]:
w = 10
h = 7.5
options(repr.plot.width = w, repr.plot.height = h)
pticksize = 40
psychcells <- WhichCells(merged_seurat, idents = c( "5"))
psych_highlight <- DimPlot(merged_seurat, reduction = 'wnn.umap',label = FALSE,cells.highlight = psychcells,cols.highlight='black',sizes.highlight=2,
              raster = NULL,pt.size=1.5,shuffle=TRUE,seed=3,label.box=FALSE,
              repel = TRUE, label.size = 4)

psych_highlight
# Save to a PDF file
ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/psych/highlight.pdf", plot = psych_highlight, width = w, height = h,dpi=300)

# # Save p3 to a PDF file
# ggsave("/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/figures/seurat/main/dimplot_main_diagnosis.pdf", plot = p3, width = w, height = h,dpi=300)

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 6)
VlnPlot(
  object = psych_cluster,
  features = 'Age',
  pt.size=0,  
  split.by='sex',
  split.plot=TRUE
  # group.by = 'seurat_clusters',
  # cols = viridis(length(unique(merged_cog$seurat_clusters)), option = 'D'),
)

options(repr.plot.width = 7, repr.plot.height = 6)
VlnPlot(
  object = psych_cluster,
  features = 'File',
  pt.size=0,  
  split.by='sex',
  split.plot=TRUE
  # group.by = 'seurat_clusters',
  # cols = viridis(length(unique(merged_cog$seurat_clusters)), option = 'D'),
)


In [ ]:
# ad <- data.frame(cluster = Idents(psych_cluster), Age=psych_cluster@meta.data$Age)
# mean_age_per_cluster <- ad %>%
#   group_by(cluster) %>%
#   summarise(mean_age = mean(Age),std_age = sd(Age),max=max(Age))
# pairwise_kruskal <- pairwise.wilcox.test(ad$Age, ad$cluster, p.adjust.method = "bonferroni")
# significant_pairs <- data.frame(from = character(), to = character(), p.value = numeric())
# print(mean_age_per_cluster)
# for (i in 1:nrow(pairwise_kruskal$p.value)) {
#   for (j in 1:ncol(pairwise_kruskal$p.value)) {
#     if (!is.na(pairwise_kruskal$p.value[i, j]) && pairwise_kruskal$p.value[i, j] < 0.05) {
#       significant_pairs <- rbind(significant_pairs, data.frame(from = rownames(pairwise_kruskal$p.value)[i],
#                                                                to = colnames(pairwise_kruskal$p.value)[j],
#                                                                p.value = pairwise_kruskal$p.value[i, j]))
#     }
#   }
# }
# significant_pairs$to = as.integer(significant_pairs$to)
# significant_pairs$dist = abs(as.integer(significant_pairs$to) - as.integer(significant_pairs$from))
# significant_pairs <- significant_pairs %>%
#   arrange(desc(dist),to)%>%
#   mutate(height = 150 - (row_number() -1) * 5)
# significant_pairs$p.star[significant_pairs$p.value < 0.05] <- "*"
# significant_pairs$p.star[significant_pairs$p.value < 0.005] <- "**"
# significant_pairs$p.star[significant_pairs$p.value < 0.0005] <- "***"
# significant_pairs#$to
# g <- VlnPlot(
#   object = psych_cluster,
#   features = 'Age',
#   pt.size=0,  
#   split.by='sex',
#   split.plot=TRUE
#   )+ scale_y_continuous(limits = c(0,150))+ geom_signif(
#     y_position = significant_pairs$height, xmin = significant_pairs$to+1, xmax = significant_pairs$from,
#     annotation = significant_pairs$p.star, tip_length = 0.02
#   )
# g

####################################################################################
# rownames(psych_cluster)
# which(rownames(psych_cluster@assays$TEMP) == 'Mania.15.45')
# which(rownames(psych_cluster@assays$TEMP) == 'Mania.30.60')
# which(rownames(psych_cluster@assays$TEMP) == 'Mania.55.85')
# GetAssayData(object = psych_cluster[["TEMP"]], slot = "data")[711,]

####################################################################################
## to do: zoek de verschillen tussen MS-SP
# psych <- data.frame(cluster = Idents(psych_cluster), diagnosis = psych_cluster$psych_cluster_diagnosis,
#                     Age=psych_cluster@meta.data$Age,
#                     manic3060=GetAssayData(object = psych_cluster[["TEMP"]], slot = "data")[714,],
#                     manic5585=GetAssayData(object = psych_cluster[["TEMP"]], slot = "data")[719,],
#                     manic1545=GetAssayData(object = psych_cluster[["TEMP"]], slot = "data")[711,])
# psych <- psych %>%
#   filter(diagnosis == "PSYCH,BP" )
# write.xlsx(psych, 'psych_mania.xlsx', row.names = TRUE)
# psych 
# result <- psych %>%
#   filter(diagnosis == "PSYCH,BP") %>%
#   group_by(cluster) %>%
#   summarise(mean_manic = mean(manic))

# print(result)
# mean_age_per_cluster <- psych %>%
#   group_by(cluster) %>%
#   summarise(mean_age = mean(Age),std_age = sd(Age))

# print(mean_age_per_cluster)
# cluster_1 <- subset(psych, cluster == "2")
# cluster_2 <- subset(psych, cluster == "1")

# # Perform Mann-Whitney U test
# mwu_result <- wilcox.test(cluster_1$Age, cluster_2$Age)

# # Summarize Mann-Whitney U test results
# print(mwu_result)

# ggplot(psych, aes(x = cluster, y = Age, fill = cluster)) +
#   geom_violin() +
#   labs(x = "Cluster", y = "Age") + theme_minimal()

####################################################################################
# VlnPlot(psych_cluster, features='Mania.15.45')
# VlnPlot(psych_cluster, features='Mania.30.60')
# VlnPlot(psych_cluster, features='Mania.65.95')
# FeaturePlot(psych_cluster, features='Mania.15.45')

####################################################################################
# cluster_markers <- FindMarkers(psych_cluster, ident.1 = 5, min.pct = 0.1)
# FeaturePlot(psych_cluster, features = 'Suicidal-ideation')

### CON cluster

In [ ]:
con_cluster <- subset(merged_seurat, idents = c(3))
Assays(con_cluster)

In [ ]:
DefaultAssay(con_cluster) <- 'FLAT'
all.symptombins <- rownames(con_cluster[["FLAT"]])
con_cluster <- NormalizeData(con_cluster,normalization.method = "LogNormalize", scale.factor = 100)
con_cluster <- ScaleData(con_cluster, features = all.symptombins,do.center = TRUE,scale.max = 10, model.use = "linear")
con_cluster <- FindVariableFeatures(con_cluster, selection.method = "vst", nfeatures = 1500)
con_cluster <- perform_pca_and_visualization(con_cluster,rname='pca')
print(ElbowPlot(con_cluster,reduction='pca'))
con_cluster <- JackStraw(con_cluster, num.replicate = 100,assay='FLAT')
con_cluster <- ScoreJackStraw(con_cluster, dims = 1:20)
print(JackStrawPlot(con_cluster, dims = 1:20))

In [ ]:
# str(con_cluster)

In [ ]:
DefaultAssay(con_cluster) <- 'TEMP'
all.symptombins <- rownames(con_cluster[["TEMP"]])
con_cluster <- NormalizeData(con_cluster,normalization.method = "LogNormalize", scale.factor = 100)
con_cluster <- ScaleData(con_cluster, features = all.symptombins,do.center = TRUE,scale.max = 10, model.use = "linear")
con_cluster <- FindVariableFeatures(con_cluster, selection.method = "vst", nfeatures = 1500)
con_cluster <- perform_pca_and_visualization(con_cluster,rname='apca') ## change to pca for plots
# print(ElbowPlot(temp,reduction='pca')) ## change to pcs for plots
# temp <- JackStraw(temp, num.replicate = 100,assay='TEMP')
# temp <- ScoreJackStraw(temp, dims = 1:20)
# print(JackStrawPlot(temp, dims = 1:20))

In [ ]:
con_cluster <- FindMultiModalNeighbors(
    con_cluster,
    reduction.list = list("pca", "apca"),
    dims.list = list(1:10, 1:20),
    k.nn = 20,
    knn.range = 200,
    modality.weight.name = list("FLAT.weight","TEMP.weight")
)

In [ ]:
con_cluster <- RunUMAP(con_cluster,
                      # dims = 1:10,
                      nn.name = "weighted.nn",
                      reduction.name = "wnn.umap",
                      reduction.key = "wnnUMAP_",
                      verbose=FALSE  
)


In [ ]:
con_cluster <- FindClusters(con_cluster,
                              resolution = 0.5,
                              graph.name = "wsnn",
                              algorithm = 3,
                              verbose = TRUE)

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 6)
p1 <- DimPlot(con_cluster, reduction = 'wnn.umap', label = TRUE, repel = TRUE, label.size = 2.5) 
p2 <- DimPlot(con_cluster, reduction = 'wnn.umap', group.by = 'simplified_diagnosis', label = TRUE, repel = TRUE, label.size = 2.5,cols=diagnosis_colors) 
p1
p2

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 6)
p_values_df <- calculate_p_values(con_cluster, diagnosis_column = 'simplified_diagnosis')
plot_cluster_diagnosis(con_cluster,p_values_df,diagnosis_column = 'simplified_diagnosis')

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 6)
VlnPlot(
  object = con_cluster,
  features = 'Age',
  # group.by = 'seurat_clusters',
  # cols = viridis(length(unique(con_cluster$seurat_clusters)), option = 'D'),
  pt.size = 0
)
VlnPlot(
  object = con_cluster,
  features = 'File',
  # group.by = 'seurat_clusters',
  # cols = viridis(length(unique(con_cluster$seurat_clusters)), option = 'D'),
  pt.size = 0
)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 12)
sup3_small <- sup3[c("ITname", "Domain", "Grouping")]
DefaultAssay(con_cluster) <- 'FLAT'
extended_cluster_markers = find_extended_cluster_markers(con_cluster,0.05,method='sign',assay='FLAT',verbose=FALSE) 
# print(extended_cluster_markers)
sub_M1 <- subset(con_cluster, cells = Cells(con_cluster), features = extended_cluster_markers)
DoHeatmap(subset(sub_M1, downsample = 100), features = extended_cluster_markers, size = 4)
generate_heatmap_plot(con_cluster,assay='FLAT', extended_cluster_markers, sup3_small,10)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 12)
sup3_small <- sup3[c("ITname", "Domain", "Grouping")]
DefaultAssay(con_cluster) <- 'TEMP'
extended_cluster_markers = find_extended_cluster_markers(con_cluster,0.05,method='sign',assay='TEMP') 
# print(extended_cluster_markers)
sub_M1 <- subset(con_cluster, cells = Cells(con_cluster), features = extended_cluster_markers)
DoHeatmap(subset(sub_M1, downsample = 100), features = extended_cluster_markers, size = 4)
generate_heatmap_plot(con_cluster,assay='TEMP', extended_cluster_markers, sup3_small,2)

# OLD

#### QC and visualizations

In [ ]:
# Visualize QC metrics as a violin plot
VlnPlot(flattened_seurat, features = c("nFeature_RNA", "nCount_RNA", "percent.dementia"), ncol = 3)
VlnPlot(temporal_seurat, features = c("nFeature_RNA", "nCount_RNA", "percent.dementia"), ncol = 3)
## nFeatureRNA: a QC metric that represents the number of detected RNA features (genes) in each cell
## here: a QC metric that represents the number of detected symptombins in each donor. so some donor have many symptombins and others fewer

## nCount_RNA:  QC metric in scRNA-seq data analysis that represents the total number of RNA molecules (counts) detected in each cell
## here: represents the total number of observations summed over all symptombins in each donor. so many donors have few observations

## percent.dementia: represents the percentage of donors that exhibit a dementia symptombin? hard to interpret?

In [ ]:
## shows the relationship between the percentages of donors expressing the "dementia" symptombins and the "executive dysfunction" symptombins
## It provides a scatter plot where each point represents a donor
plot1 <- FeatureScatter(flattened_seurat, feature1 = "percent.dementia", feature2 = "percent.executive_dysfunction")
plot2 <- FeatureScatter(flattened_seurat, feature1 = "percent.dementia", feature2 = "percent.memory")
plot1

In [ ]:
temporal_seurat <- NormalizeData(temporal_seurat, normalization.method = "LogNormalize", scale.factor = 100)
all.symptombins <- rownames(temporal_seurat)
temporal_seurat <- ScaleData(temporal_seurat, features = all.symptombins,
                do.center = TRUE, # default TRUE
                scale.max = 10, # default 10
                model.use = "linear", ## default linear
               )

In [ ]:
### OOK leuk voor als clusters er zijn!! #######
# Access the column data by column name
features <- c(most_observations_flattened)
VlnPlot(flattened_seurat, features = features, ncol = 2)

## LogNormalize of the raw observation numbers
flattened_seurat <- NormalizeData(flattened_seurat, normalization.method = "LogNormalize", scale.factor = 100)
VlnPlot(flattened_seurat, features = features, ncol = 2)

## data scaling using z-score normalization, on already normalized data.
## It standardizes the observations of symptombins within each donor based on the mean and standard deviation
all.symptombins <- rownames(flattened_seurat)
flattened_seurat <- ScaleData(flattened_seurat, 
                features = all.symptombins,
                do.center = TRUE, # default TRUE
                scale.max = 10, # default 10
                model.use = "linear", ## default linear
               )
VlnPlot(flattened_seurat, features = features, ncol = 2)

#### variable features

In [ ]:
flattened_seurat <- FindVariableFeatures(flattened_seurat, selection.method = "vst", nfeatures = 1500)
temporal_seurat <- FindVariableFeatures(temporal_seurat, selection.method = "vst", nfeatures = 1500)

In [ ]:
# M1 <- FindVariableFeatures(M1, selection.method = "vst", nfeatures = 1500)

## we can have over 1500 features (e.g. for overlapping bins). nfeatures has a large influence!
## VST is useful when you expect the variance of the features to be related to their mean expression levels. 
## I think that translates to: when you expect the variance of the symptombin to be expected to the mean lvl of the symptom bin
## e.g. would dementia, which has a higher count, also have a higher variance? I think this is the case, but we should also try the others
## mvp selects very few. dispersion (disp): selects the symptombins with the highest dispersion values.
## It focuses on identifying features with the largest variation across samples. 


print(paste("Number of Variable Features: ",length(x = VariableFeatures(object = temporal_seurat))));
# Identify the 10 most highly variable genes
top10 <- head(VariableFeatures(temporal_seurat), 20)
plot1 <- VariableFeaturePlot(temporal_seurat)
plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE)
plot2

print(paste("Number of Variable Features: ",length(x = VariableFeatures(object = flattened_seurat))));
# Identify the 10 most highly variable genes
top10 <- head(VariableFeatures(flattened_seurat), 20)
plot1 <- VariableFeaturePlot(flattened_seurat)
plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE)
plot2

#### ICA

In [ ]:
# M1 <- RunICA(object = M1, verbose = FALSE,features = VariableFeatures(object = M1),
#              nics = 50, # default is 50
#             )
# print(M1[["ica"]], dims = 1:5, nfeatures = 5)
# options(repr.plot.width = 12, repr.plot.height = 8)
# VizDimLoadings(M1, dims = 1:3, reduction = "ica")
# DimPlot(M1, reduction = "ica",dims=c(1,2),group.by = "diagnosis")
# M1 <- RunUMAP(M1, 
#               dims = 1:20,
#               reduction = 'ica', # default pca
#              )
# generate_dim_plot(M1)

### PCA

In [ ]:
flattened_seurat <- perform_pca_and_visualization(flattened_seurat)

In [ ]:
temporal_seurat <- perform_pca_and_visualization(temporal_seurat)

## cluster flattened

In [ ]:
## based on PCs computed earlie5
## uit ervaring, minder pcs geeft wat mooiere wolken, meer PCs wat 'intuitievere' plaatsing van wolken
## clustering on the nearest neighbor graph. 
## The resolution parameter determines the granularity of the clustering, with higher values resulting in more fine-grained clusters.
##  Higher values result in more clusters, while lower values yield fewer, but potentially more general clusters.

In [ ]:
flattened_seurat <- FindNeighbors(flattened_seurat, dims = 1:10,reduction='pca')
flattened_seurat <- FindClusters(flattened_seurat, reduction='pca', resolution = 0.5)

##### UMAP

In [ ]:
# If you haven't installed UMAP, you can do so via reticulate::py_install(packages =
# 'umap-learn')
flattened_seurat <- RunUMAP(flattened_seurat, dims = 1:10,reduction = 'pca',)

##### vizualise UMAP clusters

In [ ]:
generate_dim_plot(flattened_seurat)
# FeaturePlot(M1, features = "MS_PRS", reduction = "umap", cols = viridis::viridis(10))
# DimPlot(M1, reduction = "umap", group.by = "diagnosisPD")+coord_fixed(ratio = 1.5)
# DimPlot(M1, reduction = "umap", group.by = "diagnosisAD")
# DimPlot(M1, reduction = "umap", group.by = "diagnosisFTD")+coord_fixed(ratio = 1.5)
# DimPlot(M1, reduction = "umap", group.by = "diagnosisMS")
# DimPlot(M1, reduction = "umap", group.by = "clin_diagnosis")
# DimPlot(M1, reduction = "umap", group.by = "sex")

##### cluster identiy

In [ ]:
p_values_df <- calculate_p_values(flattened_seurat)
plot_cluster_diagnosis(flattened_seurat,p_values_df)

We see 6 clusters:
- 0: mainly AD
- 1: mainly PD
- 2: mainly control
- 3: mainly MS
- 4: mainly AD+FTD
- 5: mainly psych
- 6: mainly AD 

## cluster temporal

In [ ]:
temporal_seurat <- FindNeighbors(temporal_seurat, dims = 1:25,reduction='pca')
temporal_seurat <- FindClusters(temporal_seurat, reduction='pca', resolution = 0.5)

In [ ]:
temporal_seurat <- RunUMAP(temporal_seurat, dims = 1:25,reduction = 'pca',)

In [ ]:
generate_dim_plot(temporal_seurat)

In [ ]:
p_values_df <- calculate_p_values(temporal_seurat)
plot_cluster_diagnosis(temporal_seurat,p_values_df)

## project temporal on flattened

In [ ]:
flattened_clusters <- flattened_seurat@meta.data$seurat_clusters
head(colnames(flattened_seurat))
temporal_seurat$flattened_clusters <- flattened_clusters

In [ ]:
DimPlot(temporal_seurat, reduction = "umap", group.by = "flattened_clusters") +
theme(legend.key.size = unit(2, "lines"),
      legend.text = element_text(size = 14)) +
theme(plot.margin = margin(0.1, 0.1, 0.1, 0.5, "cm")) +
labs(title = "temporal_clusters flattened coloring") +
coord_fixed(ratio = 1.5)


## project flattened on temporal

In [ ]:
temporal_clusters <- temporal_seurat@meta.data$seurat_clusters
head(colnames(temporal_seurat))
flattened_seurat$temporal_clusters <- temporal_clusters

DimPlot(flattened_seurat, reduction = "umap", group.by = "temporal_clusters") +
theme(legend.key.size = unit(2, "lines"),
      legend.text = element_text(size = 14)) +
theme(plot.margin = margin(0.1, 0.1, 0.1, 0.5, "cm")) +
labs(title = "flattened_clusters temporal coloring") +
coord_fixed(ratio = 1.5)


## flattened: select AD/FTDetc.

In [ ]:
flattened_seurat_cog <- subset(flattened_seurat, idents = c(0,4,6))
flattened_seurat_cog <- NormalizeData(flattened_seurat_cog, normalization.method = "LogNormalize", scale.factor = 100)
all.symptombins <- rownames(flattened_seurat_cog)
flattened_seurat_cog <- ScaleData(flattened_seurat_cog, features = all.symptombins,
                do.center = TRUE, # default TRUE
                scale.max = 10, # default 10
                model.use = "linear", ## default linear
               )
flattened_seurat_cog <- FindVariableFeatures(flattened_seurat_cog, selection.method = "vst", nfeatures = 1500)
flattened_seurat_cog <- perform_pca_and_visualization(flattened_seurat_cog)

In [ ]:
flattened_seurat_cog <- FindNeighbors(flattened_seurat_cog, dims = 1:12,reduction='pca')
flattened_seurat_cog <- FindClusters(flattened_seurat_cog, reduction='pca', resolution = 0.5)

In [ ]:
flattened_seurat_cog <- RunUMAP(flattened_seurat_cog, dims = 1:12,reduction = 'pca',verbose=FALSE)

In [ ]:
generate_dim_plot(flattened_seurat_cog)

In [ ]:
p_values_df <- calculate_p_values(flattened_seurat_cog)
plot_cluster_diagnosis(flattened_seurat_cog,p_values_df)

#### project temporal on flattened

In [ ]:
flattened_seurat_cog$temporal_clusters_cog <- NA
matching_rows <- match(rownames(flattened_seurat_cog@meta.data), rownames(temporal_seurat_cog@meta.data))
flattened_seurat_cog$temporal_clusters_cog <- temporal_seurat_cog@meta.data$seurat_clusters[matching_rows]
# head(flattened_seurat_cog@meta.data)

DimPlot(flattened_seurat_cog, reduction = "umap", group.by = "temporal_clusters_cog") +
theme(legend.key.size = unit(2, "lines"),
      legend.text = element_text(size = 14)) +
theme(plot.margin = margin(0.1, 0.1, 0.1, 0.5, "cm")) +
labs(title = "flattened_clusters temporal coloring") +
coord_fixed(ratio = 1.5)


## temporal: select AD/FTDetc.

In [ ]:
temporal_seurat_cog <- subset(temporal_seurat, idents = c(0,3,6))
temporal_seurat_cog <- NormalizeData(temporal_seurat_cog, normalization.method = "LogNormalize", scale.factor = 100)
all.symptombins <- rownames(temporal_seurat_cog)
temporal_seurat_cog <- ScaleData(temporal_seurat_cog, features = all.symptombins,
                do.center = TRUE, # default TRUE
                scale.max = 10, # default 10
                model.use = "linear", ## default linear
               )
temporal_seurat_cog <- FindVariableFeatures(temporal_seurat_cog, selection.method = "vst", nfeatures = 1500)
temporal_seurat_cog <- perform_pca_and_visualization(temporal_seurat_cog)

In [ ]:
temporal_seurat_cog <- FindNeighbors(temporal_seurat_cog, dims = 1:20,reduction='pca')
temporal_seurat_cog <- FindClusters(temporal_seurat_cog, reduction='pca', resolution = 0.5)

In [ ]:
temporal_seurat_cog <- RunUMAP(temporal_seurat_cog, dims = 1:20,reduction = 'pca',verbose=FALSE)

In [ ]:
generate_dim_plot(temporal_seurat_cog)

In [ ]:
p_values_df <- calculate_p_values(temporal_seurat_cog)
plot_cluster_diagnosis(temporal_seurat_cog,p_values_df)

#### project flattened on temporal

In [ ]:
temporal_seurat_cog$temporal_clusters_cog <- NA
matching_rows <- match(rownames(temporal_seurat_cog@meta.data), rownames(flattened_seurat_cog@meta.data))
temporal_seurat_cog$flattened_clusters_cog <- flattened_seurat_cog@meta.data$seurat_clusters[matching_rows]
# head(flattened_seurat_cog@meta.data)

DimPlot(temporal_seurat_cog, reduction = "umap", group.by = "flattened_clusters_cog") +
theme(legend.key.size = unit(2, "lines"),
      legend.text = element_text(size = 14)) +
theme(plot.margin = margin(0.1, 0.1, 0.1, 0.5, "cm")) +
labs(title = "flattened_clusters temporal coloring") +
coord_fixed(ratio = 1.5)

### weighted nearest neighbor analysis

- : Independent preprocessing and dimensional reduction of each modality individually <br>
- : Learning cell-specific modality ‘weights’, and constructing a WNN graph that integrates the modalities <br>
- : Downstream analysis (i.e. visualization, clustering, etc.) of the WNN graph <br>

#### give clusters more informative names

In [ ]:
# M1 <- RenameIdents(object = M1,"0" = "AD","1" = "PD","2" = "CONTROL","3" = "AD.FTD","4" = "MS", "5" = "PSYCH")

In [ ]:
# head(M1@meta.data)
# hist(M1_with_PRS@meta.data$MS_PRS)

#### we can examine which symptombins are important for the current clustering 

In [ ]:
# # ## for example for MS cluster
# cluster_number <- 5

# # # # Subset the data to include only cluster 2
# # cluster03 <- subset(M1, idents = c(0,3))
# # # de_markers <- FindMarkers(cluster03, ident.1 = 0, ident.2 = 3, min.pct = 0.1)
# # # de_markers <- de_markers %>% arrange(desc(pct.1))
# # # de_markers <- de_markers %>% arrange(p_val_adj)
# # # de_markers

# # # # Perform differential expression analysis using the symptoms as features
# cluster_markers <- FindMarkers(M1, ident.1 = cluster_number, min.pct = 0.1)
# cluster_markers <- cluster_markers %>% arrange(desc(avg_log2FC))
# # cluster_markers <- head(cluster_markers %>% arrange(desc(pct.1)),5)
# # cluster_markers <- cluster_markers %>% filter(p_val_adj < 0.05)
# cluster_markers <- head(cluster_markers %>% arrange(p_val_adj))
# head(cluster_markers)


#### we can then vizualise these symtpombins individually, or together

In [ ]:
# # ## MS cluster
# # features = c('Muscle.Weakness.0.30BD','Muscle.Weakness.35.65','Dementia.0.10BD','Dementia.65.95')

# # ## PD cluster
# # features = c('Parkinsonism.0.10BD','Parkinsonism.60.90','Parkinsonism.5.15BD')

# FeaturePlot(M1, features = rownames(cluster_markers),reduction = "umap",ncol=3)
# # FeaturePlot(M1, features = c('Dementia.60.90','Dementia.0.10BD'), blend = TRUE)

#### We can do this for all clusters and make a heatmap of the n most important features

In [ ]:
quant_features <- colnames(flattened_data)
quant_features <- quant_features[quant_features != "DonorID"]
quant_features <- as.character(quant_features)

In [ ]:
sup3_small <- sup3[c("ITname", "Domain", "Grouping")]
# head(sup3_small,30)

In [ ]:
sup3_small <- sup3[c("ITname", "Domain", "Grouping")]
extended_cluster_markers = find_extended_cluster_markers(M1,1e-20,method='sign',quant=TRUE) 
# print(extended_cluster_markers)
sub_M1 <- subset(M1, cells = Cells(M1), features = extended_cluster_markers)
DoHeatmap(subset(sub_M1, downsample = 100), features = extended_cluster_markers, size = 4)
generate_heatmap_plot(M1, extended_cluster_markers, sup3_small,3)

In [ ]:
extended_cluster_markers = find_extended_cluster_markers(M1,10,method='topn',quant=FALSE) 
# print(extended_cluster_markers)
sub_M1 <- subset(M1, cells = Cells(M1), features = extended_cluster_markers)
DoHeatmap(subset(sub_M1, downsample = 100), features = extended_cluster_markers, size = 4)
generate_heatmap_plot(M1, extended_cluster_markers, sup3_small,3)

#### rerun for specific clusters

In [ ]:
M1_filtered <- subset(M1, idents = c(0,4,6)) ## AD
selected_donorIDs <- colnames(M1_filtered)
# selected_donorIDs <- gsub(" ", "", selected_donorIDs)
head(selected_donorIDs)

In [ ]:
merged_data <- merge(age_reshaped, flattened_data, by = "DonorID", all = TRUE)
# merged_data <- age_reshaped
# merged_data <- flattened_data
rownames(merged_data) <- merged_data$DonorID ##set rownames
head(rownames(merged_data))
dim(merged_data)
cog_clust <- merged_data[rownames(merged_data) %in% selected_donorIDs, ]

# head(cog_clust)
cog_clust$DonorID <- NULL
cog_clust$Age <-NULL
dim(cog_clust)
cog_clust = t(cog_clust)
M2 <- CreateSeuratObject(counts = cog_clust, project = "NND", min.cells = 10, min.features = 1)

M2
# calculates the percentage of donors with symptombins related to dementia and adds this information as a new metadata column
M2[["percent.dementia"]] <- PercentageFeatureSet(M2, pattern = "Dementia")
M2[["percent.executive_dysfunction"]] <- PercentageFeatureSet(M2, pattern = "Execut")
M2[["percent.memory"]] <- PercentageFeatureSet(M2, pattern = "Memory")

##  assigns the sex information from the external metadata table (meta_data) to the Seurat object
M2[['sex']] = meta_data[match(names(M2$orig.ident), rownames(meta_data)),'sex']
M2[['Age_bin']] = meta_data[match(names(M2$orig.ident), rownames(meta_data)),'age_bin']
M2[['Age']] = meta_data[match(names(M2$orig.ident), rownames(meta_data)),'age_at_death']
M2[['perfect_diagnosis']] = meta_data[match(names(M2$orig.ident), rownames(meta_data)),'perfect_diagnosis']
M2[['wrong_diagnosis']] = meta_data[match(names(M2$orig.ident), rownames(meta_data)),'wrong_diagnosis']
M2[['diagnosis']] <- meta_data[match(names(M2$orig.ident), rownames(meta_data)),'neuropathological_diagnosis']



# ## LogNormalize of the raw observation numbers
M2 <- NormalizeData(M2, normalization.method = "LogNormalize", scale.factor = 100)

# ## data scaling using z-score normalization, on already normalized data.
# ## It standardizes the observations of symptombins within each donor based on the mean and standard deviation
all.symptombins <- rownames(M2)
M2 <- ScaleData(M2, 
                features = all.symptombins,
                do.center = TRUE, # default TRUE
                scale.max = 10, # default 10
                model.use = "linear", ## default linear
               )
M2 <- FindVariableFeatures(M2, selection.method = "vst", nfeatures = 1500)
M2 <- RunPCA(object = M2, verbose = FALSE,features = VariableFeatures(object = M2),
             npcs = 50, # default is 50
            )
M2 <- JackStraw(M2, num.replicate = 100)
M2 <- ScoreJackStraw(M2, dims = 1:20)
JackStrawPlot(M2, dims = 1:20)
ElbowPlot(M2)




In [ ]:
M2 <- FindNeighbors(M2, dims = 1:15)
M2 <- FindClusters(M2, resolution = 0.5) ## defualt is 0.8)
M2 <- RunUMAP(M2,dims = 1:15,reduction = 'pca') # default pca

In [ ]:
generate_dim_plot(M2)

In [ ]:
p_values_df <- calculate_p_values(M2)
plot_cluster_diagnosis(M2,p_values_df)


In [ ]:
extended_cluster_markers = find_extended_cluster_markers(M2,1e-20,method='sign') 
sub_M2 <- subset(M2, cells = Cells(M2), features = extended_cluster_markers)
DoHeatmap(subset(sub_M2, downsample = 100), features = extended_cluster_markers, size = 4)
generate_heatmap_plot(M2, extended_cluster_markers, sup3_small,6)

In [ ]:
extended_cluster_markers = find_extended_cluster_markers(M2,0.01,method='sign',quant=TRUE) 
sub_M2 <- subset(M2, cells = Cells(M2), features = extended_cluster_markers)
DoHeatmap(subset(sub_M2, downsample = 100), features = extended_cluster_markers, size = 4)

### others

In [ ]:
# M1_filtered <- subset(M1, idents = c(0,1,2,3,4,6,7,8,9))
## e.g. is there any clustering within the MS cluster?
M1_filtered <- subset(M1, idents = c(0,1,3)) ## AD
# M1_filtered <- subset(M1, idents = c(1)) ## PD
M1_filtered
M1_filtered <- NormalizeData(M1_filtered, normalization.method = "LogNormalize", scale.factor = 100)
M1_filtered <- ScaleData(M1_filtered, features = all.symptombins)
M1_filtered <- FindVariableFeatures(M1_filtered, selection.method = "vst", nfeatures = 1500)
M1_filtered <- RunPCA(M1_filtered, features = VariableFeatures(object = M1_filtered),verbose=FALSE)
M1_filtered <- JackStraw(M1_filtered, num.replicate = 100)
M1_filtered <- ScoreJackStraw(M1_filtered, dims = 1:20)

JackStrawPlot(M1_filtered, dims = 1:20)
ElbowPlot(M1_filtered)

In [ ]:
M1_filtered <- FindNeighbors(M1_filtered, dims = 1:15)
M1_filtered <- FindClusters(M1_filtered, resolution = 0.3)
M1_filtered <- RunUMAP(M1_filtered, dims = 1:15,verbose=FALSE)

In [ ]:
generate_dim_plot(M1_filtered)

In [ ]:
p_values_df <- calculate_p_values(M1_filtered)
plot_cluster_diagnosis(M1_filtered,p_values_df)

In [ ]:
extended_cluster_markers = find_extended_cluster_markers(M1_filtered,0.01,method='sign',quant=TRUE) 
sub_M1 <- subset(M1_filtered, cells = Cells(M1_filtered), features = extended_cluster_markers)
DoHeatmap(subset(sub_M1, downsample = 100), features = extended_cluster_markers, size = 4)
generate_heatmap_plot(M1_filtered, extended_cluster_markers, sup3_small,3)

In [ ]:
extended_cluster_markers = find_extended_cluster_markers(M1_filtered,20,method='topn') 
sub_M1 <- subset(M1_filtered, cells = Cells(M1_filtered), features = extended_cluster_markers)
DoHeatmap(subset(sub_M1, downsample = 100), features = extended_cluster_markers, size = 4)
generate_heatmap_plot(M1_filtered, extended_cluster_markers, sup3_small,3)

In [ ]:
FTD_cluster <- subset(M1_filtered, idents = c(1)) ## AD
FTDdonors <- colnames(FTD_cluster)
head(FTDdonors)

FTD_other <-subset(M1_filtered, idents = c(0,2,3)) ## AD
FTD_other <- colnames(FTD_other)
head(FTD_other)

In [ ]:
meta_data_FTD <- meta_data %>%
  filter(neuropathological_diagnosis == "FTD")

meta_data_FTD <- meta_data_FTD %>%
  mutate(FTDcluster = ifelse(rownames(meta_data_FTD) %in% FTDdonors, 1, 0))

grouped_data <- meta_data_FTD %>%
  group_by(FTDcluster)
# head(meta_data_FTD)
# Calculate the count for each column within each group
summary_data <- grouped_data %>%
  summarise(
    count_perfect_diagnosis = sum(perfect_diagnosis),
    count_medium_diagnosis = sum(medium_diagnosis),
    count_wrong_diagnosis = sum(wrong_diagnosis)
  )

print(summary_data)


# Assuming your tibble is named 'summary_data', you can do the following:
summary_data <- summary_data %>%
  rowwise() %>%
  mutate(
    percent_perfect_diagnosis = count_perfect_diagnosis / sum(c(count_perfect_diagnosis, count_medium_diagnosis, count_wrong_diagnosis)) * 100,
    percent_medium_diagnosis = count_medium_diagnosis / sum(c(count_perfect_diagnosis, count_medium_diagnosis, count_wrong_diagnosis)) * 100,
    percent_wrong_diagnosis = count_wrong_diagnosis / sum(c(count_perfect_diagnosis, count_medium_diagnosis, count_wrong_diagnosis)) * 100
  ) %>%
  select(-starts_with("count_"))  # Remove the count columns



# Print the updated tibble with percentages
print(summary_data)
melted_data <- reshape2::melt(summary_data, id.vars = "FTDcluster")

# Create a grouped bar chart
ggplot(melted_data, aes(x = factor(FTDcluster), y = value, fill = variable)) +
  geom_bar(stat = "identity", position = "dodge") +
  labs(title = "Percentage of Diagnosis by FTDcluster",
       x = "FTDcluster",
       y = "Percentage (%)") +
  scale_fill_discrete(name = "Diagnosis Type") +
  theme_minimal()

In [ ]:
features = c('Compulsive.behavior')
FeaturePlot(M1_filtered, features = features,reduction = "umap",ncol=3)

#### vizualise PRS scores on the different clusters
hypothesis: 
- cluster 0, which has mainly AD donors, might have a higher PRS score for AD as well
- cluster 3, which has mainly FTD donors, might have a higher PRS score for FTD as well
- cluster 4, which has mainly MS donors, might have a higher PRS score for MS as well

In [ ]:
M1_with_PRS <- subset(M1, subset = (MS_PRS != 'exclude' & AD_PRS != 'exclude' & FTD_PRS != 'exclude' ))
M1_with_PRS$MS_PRS <- as.numeric(M1_with_PRS$MS_PRS)
M1_with_PRS$AD_PRS <- as.numeric(M1_with_PRS$AD_PRS)
M1_with_PRS$FTD_PRS <- as.numeric(M1_with_PRS$FTD_PRS)
M1_with_PRS@meta.data$seurat_clusters <- factor(M1_with_PRS@meta.data$seurat_clusters,
                                               levels = c("0", "1", "2", "3", "4", "5"),
                                               labels = c("AD", "PD", "CONTROL", "AD.FTD", "MS", "PSYCH"))

VlnPlot(M1_with_PRS, features = c("AD_PRS","FTD_PRS","MS_PRS"), ncol = 3)
DotPlot(M1_with_PRS, features = c("MS_PRS","AD_PRS","FTD_PRS"),
        scale.min=0.9999,
        scale.max=1,
        dot.scale=10,
        col.min = -5,
        col.max = 5,
        cols = 'RdGy'
       ) + 
RotatedAxis() +
coord_flip()
## average expression: indicates the average number of observations of a particular symptombin within each cluster, or in this case average PRS score. 
## Percent Expressed: This value represents the percentage of donors within each cluster that have observations for that symptombin, or in this case PRS. 
## It provides an estimation of the proportion of donors in the cluster where the symptombin is detected or expressed. 
## The percentage is calculated by dividing the number of donors with the symptombin by the total number of donors in the cluster and multiplying by 100.


cluster_identity <- M1_with_PRS@meta.data$seurat_clusters
# cluster_identity <- Idents(M1)
ms_prs <- M1_with_PRS@meta.data$MS_PRS
ad_prs <- M1_with_PRS@meta.data$AD_PRS
ftd_prs <- M1_with_PRS@meta.data$FTD_PRS
temp <- data.frame(Cluster = cluster_identity, MS_PRS = ms_prs, AD_PRS = ad_prs,  FTD_PRS = ftd_prs)
print(head(temp,10))
print(table(temp$Cluster))
average_values <- temp %>%
  group_by(Cluster) %>%
  summarize(
    mean_MS_PRS = mean(MS_PRS),
    mean_AD_PRS = mean(AD_PRS),
    mean_FTD_PRS = mean(FTD_PRS)
  )
print(average_values)
heatmap_data <- as.data.frame(average_values)
heatmap_data_long <- reshape2::melt(heatmap_data, id.vars = "Cluster")
heatmap_plot <- ggplot(heatmap_data_long, aes(x = variable, y = Cluster, fill = value)) +
  geom_tile() +
  scale_fill_viridis_c() +  # Use the viridis color palette
  labs(x = "Clusters", y = "Markers")  +coord_flip()# Set x and y axis labels

print(heatmap_plot)

### within the MS cluster we see two subgroups, seemingly based on age (and when symtpoms first occur)

#### cluster identity

#### what drives the subclusters?

In [ ]:
meta <- as.data.frame(M1_filtered@meta.data$MS_PRS,row.names = rownames(M1_filtered@meta.data))
UMAP_test <- as.data.frame(Embeddings(M1_filtered,reduction='umap'))
head(UMAP_test)
# head(meta)
linear_model <- lm(UMAP_2 ~ UMAP_1, data = UMAP_test)  # Perform linear regression
scatter_plot <- ggplot(data = UMAP_test, aes(x = UMAP_1, y = UMAP_2)) +
  geom_point()  # Add points to the plot

slope <- coef(linear_model)[["UMAP_1"]]  # Extract the slope
intercept <- coef(linear_model)[["(Intercept)"]]  # Extract the intercept
regression_line <- geom_abline(slope = slope, intercept = intercept, color = "red", linetype = "dashed")  # Add regression line

scatter_plot + regression_line


# Define x-coordinates for the two points
x1 <- -3
x2 <- 3

# Calculate the corresponding y-coordinates using the regression line equation
y1 <- slope * x1 + intercept
y2 <- slope * x2 + intercept

# Print the coordinates of the two points on the line
point1 <- c(x1, y1)
point2 <- c(x2, y2)

p4 <- function(p1, p2, p3) {
  x1 <- p1[1]
  y1 <- p1[2]
  x2 <- p2[1]
  y2 <- p2[2]
  x3 <- p3[1]
  y3 <- p3[2]
  dx <- x2 - x1
  dy <- y2 - y1
  det <- dx*dx + dy*dy
  a <- (dy * (y3 - y1) + dx * (x3 - x1)) / det
  return(list(x1 + a * dx, y1 + a * dy))
}


# p4(point1,point2,c(0.8231990,-0.1644701))
UMAP_test$x <- apply(UMAP_test,1,function(row){p4(c(-1.0, 1.0), c(1.0, -1.0), c(row["UMAP_1"], row["UMAP_2"])) })
UMAP_test <- cbind(UMAP_test, do.call(rbind, UMAP_test$x))
colnames(UMAP_test)[4:5] <- c("x_coordinate", "y_coordinate")


# Plot the values in the "x" column
# Create an empty plot
plot(NA, xlim = range(UMAP_test$UMAP_1), ylim = range(UMAP_test$UMAP_2), xlab = "UMAP_1", ylab = "UMAP_2")

# Plot the points
for (i in 1:nrow(UMAP_test)) {
  points(UMAP_test$UMAP_1[i], UMAP_test$UMAP_2[i], col = "blue")  # Plot the original points
  points(UMAP_test$x[[i]][1], UMAP_test$x[[i]][2], col = "red")  # Plot the points from the "x" column
}
UMAP_test$x_coordinate <- sapply(UMAP_test$x_coordinate, `[[`, 1)
UMAP_test$y_coordinate <- sapply(UMAP_test$y_coordinate, `[[`, 1)
# Add a legend
legend("topright", legend = c("Original Points", "Points from x"), col = c("blue", "red"), pch = 1)

UMAP_test$y2 <- UMAP_test$y_coordinate - UMAP_test$y_coordinate
UMAP_test <- merge(UMAP_test, meta, by = "row.names", all = TRUE)
colnames(UMAP_test)[8] <- "PRS"

UMAP_test <- UMAP_test[UMAP_test$PRS != 'exclude', ]
UMAP_test$PRS <- as.numeric(UMAP_test$PRS)
head(UMAP_test,10)
linear_model <- lm(PRS ~ x_coordinate, data = UMAP_test)  # Perform linear regression
summary(linear_model)
scatter_plot <- ggplot(data = UMAP_test, aes(x = x_coordinate, y = PRS)) +
  geom_point()  # Add points to the plot

slope <- coef(linear_model)[["x_coordinate"]]  # Extract the slope
intercept <- coef(linear_model)[["(Intercept)"]]  # Extract the intercept
regression_line <- geom_abline(slope = slope, intercept = intercept, color = "red", linetype = "dashed")  # Add regression line

scatter_plot + regression_line

In [ ]:
# M1_MS <- subset(M1_filtered, subset = (MS_PRS > 0 | MS_PRS < 0 | MS_PRS == 0))
# cluster_identity <- M1_MS@meta.data$seurat_clusters
# ms_prs <- M1_MS@meta.data$MS_PRS
# ad_prs <- M1_MS@meta.data$AD_PRS
# ftd_prs <- M1_MS@meta.data$FTD_PRS
# temp <- data.frame(Cluster = cluster_identity, MS_PRS = ms_prs, AD_PRS = ad_prs,  FTD_PRS = ftd_prs)
# head(temp)

# average_values <- temp %>%
#   group_by(Cluster) %>%
#   summarize(
#     mean_MS_PRS = mean(MS_PRS),
#     mean_AD_PRS = mean(AD_PRS),
#     mean_FTD_PRS = mean(FTD_PRS)
#   )
# print(average_values)
# heatmap_data <- as.data.frame(average_values)
# # rownames(heatmap_data) <- NULL
# heatmap_data_long <- reshape2::melt(heatmap_data, id.vars = "Cluster")
# print(heatmap_data)
# print(heatmap_data_long)
# heatmap_plot <- ggplot(heatmap_data_long, aes(x = variable, y = Cluster, fill = value)) +
#   geom_tile() +
#   scale_fill_viridis_c() +  # Use the viridis color palette
#   labs(x = "Clusters", y = "Markers")  # Set x and y axis labels
# print(heatmap_plot)

#### are there differences in MS PRS between the two groups?

In [ ]:
M1_with_PRS <- subset(M1_filtered, subset = (MS_PRS != 'exclude'))
M1_with_PRS$MS_PRS <- as.numeric(M1_with_PRS$MS_PRS)
M1_with_PRS@meta.data$seurat_clusters <- factor(M1_with_PRS@meta.data$seurat_clusters,
                                               levels = c("0", "1"))

VlnPlot(M1_with_PRS, features = c("MS_PRS"), ncol = 1)
DotPlot(M1_with_PRS, features = c("MS_PRS"),
        scale.min=0,
        scale.max=1,
        # dot.scale=10,
        col.min = 0,
        col.max = 1,
        cols = 'RdGy'
       ) + 
RotatedAxis() +
coord_flip()

cluster_identity <- M1_with_PRS@meta.data$seurat_clusters
# cluster_identity <- Idents(M1)
ms_prs <- M1_with_PRS@meta.data$MS_PRS
temp <- data.frame(Cluster = cluster_identity, MS_PRS = ms_prs)
# print(head(temp,10))
cluster_0 <- temp$MS_PRS[temp$Cluster == 0]
cluster_1 <- temp$MS_PRS[temp$Cluster == 1]

# Perform t-test
t_test_result <- t.test(cluster_0, cluster_1)

# Print the t-test result
print(t_test_result)
# print(table(temp$Cluster))
# average_values <- temp %>%
#   group_by(Cluster) %>%
#   summarize(
#     mean_MS_PRS = mean(MS_PRS)
#   )
# print(average_values)
# heatmap_data <- as.data.frame(average_values)
# heatmap_data_long <- reshape2::melt(heatmap_data, id.vars = "Cluster")
# heatmap_plot <- ggplot(heatmap_data_long, aes(x = variable, y = Cluster, fill = value)) +
#   geom_tile() +
#   scale_fill_viridis_c(limits = c(0, 1)) +  # Set the color range limits
#   labs(x = "Clusters", y = "Markers") +
#   coord_flip()

# print(heatmap_plot)


In [ ]:
VlnPlot(M1_filtered, features = features, ncol = 2)
# FeaturePlot(M1_filtered, features = "MS_PRS", reduction = "umap", cols = viridis::viridis(10))

In [ ]:
cluster_number <- 5

# Subset the data to include only cluster 2
# cluster2 <- subset(M1, ident.1 == cluster_number)

# Perform differential expression analysis using the symptoms as features
cluster2_markers <- FindMarkers(M1_filtered, ident.1 = cluster_number, min.pct = 0.1)
cluster2_markers <- cluster2_markers %>% arrange(desc(avg_log2FC))
cluster2_markers <- cluster2_markers %>% arrange(pct.1)
cluster2_markers <- cluster2_markers %>% arrange(p_val_adj)
cluster2_markers